In [36]:
import pandas as pd
import numpy as np
import string  
import nltk
import re
#import scipy
#import matplotlib.pyplot as plt
#import urllib 
import sklearn
from sklearn import preprocessing
#from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import mean_squared_error, f1_score

!pip install catboost 
from catboost import CatBoostRegressor, CatBoostClassifier
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

display(train)
display(test)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [37]:
#Rellenando nulos
train['keyword'].fillna('no keyword', inplace = True) 
train['keyword'] = train['keyword'].str.replace('%20', ' ')
train['location'].fillna('no location', inplace = True)

test['keyword'].fillna('no keyword', inplace = True) 
test['keyword'] = test['keyword'].str.replace('%20', ' ')
test['location'].fillna('no location', inplace = True)

train['length'] = train['text'].str.len()
test['length'] = test['text'].str.len()

train.head()

,id,keyword,location,text,target,length
0,1,no keyword,no location,Our Deeds are the Reason of this #earthquake M...,1,69
1,4,no keyword,no location,Forest fire near La Ronge Sask. Canada,1,38
2,5,no keyword,no location,All residents asked to 'shelter in place' are ...,1,133
3,6,no keyword,no location,"13,000 people receive #wildfires evacuation or...",1,65
4,7,no keyword,no location,Just got sent this photo from Ruby #Alaska as ...,1,88


In [38]:
#limpiando el texto

import nltk
nltk.download('stopwords') 
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

#FUNCIONES UTILES

def only_letters(tweet):
    tweet = re.sub(r'http\S*', '', tweet)
    tweet = re.sub(r'[^a-z\s]', '', tweet)
    return tweet

stop_words=set(stopwords.words("english"))

def filter_stopwords(tokenized_text):
    filtered_words=[]
    for w in tokenized_text:
        if w not in stop_words:
            filtered_words.append(w)
    return filtered_words

lemmatizer = WordNetLemmatizer()

def lemmatize_tweet(tweet):
    lemmatized_words = []
    for word in tweet:
        lemmatized_words.append(lemmatizer.lemmatize(word))
    return lemmatized_words

def transform_to_text(tweet_words):
    return " ".join(tweet_words)

#Cleaning text

train['clean_text'] = train['text'].str.lower()

train['clean_text'] = train['clean_text'].apply(only_letters)    

test['clean_text'] = test['text'].str.lower()

test['clean_text'] = test['clean_text'].apply(only_letters) 

#Tokenización

train['clean_text'] = train['clean_text'].apply(word_tokenize)

test['clean_text'] = test['clean_text'].apply(word_tokenize)

#Remove stopwords

train['clean_text'] = train['clean_text'].apply(filter_stopwords) 

test['clean_text'] = test['clean_text'].apply(filter_stopwords) 


#Lemmatization                                                                       

train['clean_text'] = train['clean_text'].apply(lemmatize_tweet)  

train['clean_text'] = train['clean_text'].apply(transform_to_text)

test['clean_text'] = test['clean_text'].apply(lemmatize_tweet)  

test['clean_text'] = test['clean_text'].apply(transform_to_text)

display(train)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hijos.ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hijos.ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hijos.ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,id,keyword,location,text,target,length,clean_text
0,1,no keyword,no location,Our Deeds are the Reason of this #earthquake M...,1,69,deed reason earthquake may allah forgive u
1,4,no keyword,no location,Forest fire near La Ronge Sask. Canada,1,38,forest fire near la ronge sask canada
2,5,no keyword,no location,All residents asked to 'shelter in place' are ...,1,133,resident asked shelter place notified officer ...
3,6,no keyword,no location,"13,000 people receive #wildfires evacuation or...",1,65,people receive wildfire evacuation order calif...
4,7,no keyword,no location,Just got sent this photo from Ruby #Alaska as ...,1,88,got sent photo ruby alaska smoke wildfire pour...
...,...,...,...,...,...,...,...
7608,10869,no keyword,no location,Two giant cranes holding a bridge collapse int...,1,83,two giant crane holding bridge collapse nearby...
7609,10870,no keyword,no location,@aria_ahrary @TheTawniest The out of control w...,1,125,ariaahrary thetawniest control wild fire calif...
7610,10871,no keyword,no location,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,65,utckm volcano hawaii
7611,10872,no keyword,no location,Police investigating after an e-bike collided ...,1,137,police investigating ebike collided car little...


In [39]:
#agregando algunos features

keywords_list = list(set(train.keyword))  #todas las keywords únicas

locations_list = list(set(train.location)) #todas las locaciones únicas
locations_list.remove('no location')

def amount_keywords(tweet):
    cont = 0
    for keyword in keywords_list:
        if keyword in tweet:
            cont += 1
    return cont

train['amount_words'] = train['clean_text'].str.split(' ').apply(lambda x: len(x))  

test['amount_words'] = test['clean_text'].str.split(' ').apply(lambda x: len(x))  

train['amount_keywords_in_tweet'] = train['clean_text'].apply(lambda x: amount_keywords(x))

test['amount_keywords_in_tweet'] = test['clean_text'].apply(lambda x: amount_keywords(x))

def amount_locations(tweet):    #esta función podría unificarse con la de keywords pasandole por parámetro la lista a recorrer
    cont = 0
    for location in locations_list:
        if location in tweet:
            cont += 1
    return cont

train['amount_locations_in_tweet'] = train['clean_text'].apply(lambda x: amount_locations(x))

test['amount_locations_in_tweet'] = test['clean_text'].apply(lambda x: amount_locations(x))

In [40]:
#más features
def get_hashtags(s):
    return list(part[1:] for part in s.split() if part.startswith('#'))

train['hashtags'] = train['text'].apply(get_hashtags)
test['hashtags'] = test['text'].apply(get_hashtags)

def hashtag_length_proportion(hashtags, length):
    return len(''.join(hashtags))/length

train['len_hash_over_text'] = train.apply(lambda data: hashtag_length_proportion(data['hashtags'], data['length']), axis=1)
test['len_hash_over_text'] = test.apply(lambda data: hashtag_length_proportion(data['hashtags'], data['length']), axis=1)

def get_mentioned_users(s):
    return list(part[1:] for part in s.split() if part.startswith('@'))

train['users'] = train['text'].apply(get_mentioned_users)
test['users'] = test['text'].apply(get_mentioned_users)

def amount_hashtags(l):
    return len(l)

train['amount_hashtags'] = train['hashtags'].apply(amount_hashtags)
test['amount_hashtags'] = test['hashtags'].apply(amount_hashtags)

def amount_users(l):
    return len(l)

def has_url(text):
    return int('http' in text)

train['has_url'] = train['text'].apply(has_url)
test['has_url'] = test['text'].apply(has_url)

def find_url(text):
    urls = re.findall(r'(https?://\S+)', text)
    return urls

train['urls'] = train['text'].apply(find_url)
test['urls'] = test['text'].apply(find_url)
train['amount_urls'] = train['urls'].apply(lambda x: len(x))
test['amount_urls'] = test['urls'].apply(lambda x: len(x))

train['amount_users'] = train['users'].apply(amount_users)
test['amount_users'] = test['users'].apply(amount_users)


In [8]:
#construyendo los datos de entrenamiento y de test
selected_df = train[['length','amount_words','amount_keywords_in_tweet','amount_locations_in_tweet', 'amount_urls','amount_users', 'amount_hashtags','len_hash_over_text', 'has_url']] 

X, y = selected_df,train['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [17]:
#Pruebo primero con CatBoostRegressor
#CatBoostRegressor por defecto
model = CatBoostRegressor(random_seed = 41)
model.fit(X_train, y_train)
preds = model.predict(X_test)

Learning rate set to 0.053835
0:	learn: 0.4908195	total: 4.38ms	remaining: 4.37s
1:	learn: 0.4868907	total: 8.46ms	remaining: 4.22s
2:	learn: 0.4835794	total: 12.6ms	remaining: 4.18s
3:	learn: 0.4806080	total: 16.7ms	remaining: 4.16s
4:	learn: 0.4780186	total: 20.7ms	remaining: 4.12s
5:	learn: 0.4755176	total: 24ms	remaining: 3.98s
6:	learn: 0.4731890	total: 27.4ms	remaining: 3.89s
7:	learn: 0.4708731	total: 31.7ms	remaining: 3.93s
8:	learn: 0.4690652	total: 34.6ms	remaining: 3.81s
9:	learn: 0.4672509	total: 38.7ms	remaining: 3.83s
10:	learn: 0.4656755	total: 42.7ms	remaining: 3.84s
11:	learn: 0.4640961	total: 46.7ms	remaining: 3.85s
12:	learn: 0.4626838	total: 50.8ms	remaining: 3.86s
13:	learn: 0.4614090	total: 54.9ms	remaining: 3.87s
14:	learn: 0.4600285	total: 59.1ms	remaining: 3.88s
15:	learn: 0.4587520	total: 63.1ms	remaining: 3.88s
16:	learn: 0.4576459	total: 67.3ms	remaining: 3.89s
17:	learn: 0.4565588	total: 71.4ms	remaining: 3.9s
18:	learn: 0.4556274	total: 75.5ms	remaining: 3

170:	learn: 0.4305049	total: 837ms	remaining: 4.06s
171:	learn: 0.4304763	total: 843ms	remaining: 4.06s
172:	learn: 0.4304112	total: 848ms	remaining: 4.05s
173:	learn: 0.4303534	total: 854ms	remaining: 4.05s
174:	learn: 0.4302696	total: 870ms	remaining: 4.1s
175:	learn: 0.4302252	total: 876ms	remaining: 4.1s
176:	learn: 0.4301808	total: 882ms	remaining: 4.1s
177:	learn: 0.4300727	total: 886ms	remaining: 4.09s
178:	learn: 0.4299012	total: 891ms	remaining: 4.09s
179:	learn: 0.4298362	total: 896ms	remaining: 4.08s
180:	learn: 0.4296536	total: 915ms	remaining: 4.14s
181:	learn: 0.4296039	total: 919ms	remaining: 4.13s
182:	learn: 0.4294512	total: 924ms	remaining: 4.13s
183:	learn: 0.4293665	total: 931ms	remaining: 4.13s
184:	learn: 0.4293208	total: 936ms	remaining: 4.12s
185:	learn: 0.4292576	total: 942ms	remaining: 4.12s
186:	learn: 0.4291487	total: 948ms	remaining: 4.12s
187:	learn: 0.4290837	total: 954ms	remaining: 4.12s
188:	learn: 0.4290320	total: 959ms	remaining: 4.11s
189:	learn: 0.4

349:	learn: 0.4171146	total: 1.82s	remaining: 3.38s
350:	learn: 0.4170993	total: 1.82s	remaining: 3.37s
351:	learn: 0.4170296	total: 1.83s	remaining: 3.37s
352:	learn: 0.4169507	total: 1.83s	remaining: 3.36s
353:	learn: 0.4168513	total: 1.84s	remaining: 3.36s
354:	learn: 0.4167911	total: 1.85s	remaining: 3.36s
355:	learn: 0.4167014	total: 1.85s	remaining: 3.35s
356:	learn: 0.4166459	total: 1.86s	remaining: 3.35s
357:	learn: 0.4165743	total: 1.86s	remaining: 3.34s
358:	learn: 0.4165271	total: 1.87s	remaining: 3.34s
359:	learn: 0.4164773	total: 1.87s	remaining: 3.33s
360:	learn: 0.4164068	total: 1.88s	remaining: 3.32s
361:	learn: 0.4163803	total: 1.88s	remaining: 3.31s
362:	learn: 0.4162936	total: 1.89s	remaining: 3.31s
363:	learn: 0.4162607	total: 1.89s	remaining: 3.3s
364:	learn: 0.4162378	total: 1.89s	remaining: 3.29s
365:	learn: 0.4161847	total: 1.9s	remaining: 3.29s
366:	learn: 0.4161019	total: 1.9s	remaining: 3.28s
367:	learn: 0.4160357	total: 1.9s	remaining: 3.27s
368:	learn: 0.41

525:	learn: 0.4074637	total: 2.65s	remaining: 2.39s
526:	learn: 0.4073959	total: 2.65s	remaining: 2.38s
527:	learn: 0.4073354	total: 2.66s	remaining: 2.38s
528:	learn: 0.4072885	total: 2.66s	remaining: 2.37s
529:	learn: 0.4072809	total: 2.67s	remaining: 2.36s
530:	learn: 0.4072246	total: 2.67s	remaining: 2.36s
531:	learn: 0.4072054	total: 2.67s	remaining: 2.35s
532:	learn: 0.4071934	total: 2.68s	remaining: 2.35s
533:	learn: 0.4071529	total: 2.68s	remaining: 2.34s
534:	learn: 0.4071255	total: 2.69s	remaining: 2.34s
535:	learn: 0.4070785	total: 2.69s	remaining: 2.33s
536:	learn: 0.4070224	total: 2.7s	remaining: 2.33s
537:	learn: 0.4069904	total: 2.71s	remaining: 2.32s
538:	learn: 0.4069657	total: 2.71s	remaining: 2.32s
539:	learn: 0.4069465	total: 2.71s	remaining: 2.31s
540:	learn: 0.4068997	total: 2.72s	remaining: 2.31s
541:	learn: 0.4068396	total: 2.72s	remaining: 2.3s
542:	learn: 0.4068345	total: 2.73s	remaining: 2.29s
543:	learn: 0.4067876	total: 2.73s	remaining: 2.29s
544:	learn: 0.

689:	learn: 0.3999873	total: 3.48s	remaining: 1.56s
690:	learn: 0.3999406	total: 3.48s	remaining: 1.56s
691:	learn: 0.3998636	total: 3.49s	remaining: 1.55s
692:	learn: 0.3998354	total: 3.49s	remaining: 1.55s
693:	learn: 0.3997625	total: 3.5s	remaining: 1.54s
694:	learn: 0.3997516	total: 3.5s	remaining: 1.54s
695:	learn: 0.3996609	total: 3.51s	remaining: 1.53s
696:	learn: 0.3996270	total: 3.51s	remaining: 1.53s
697:	learn: 0.3996092	total: 3.52s	remaining: 1.52s
698:	learn: 0.3995984	total: 3.52s	remaining: 1.52s
699:	learn: 0.3995530	total: 3.53s	remaining: 1.51s
700:	learn: 0.3995067	total: 3.53s	remaining: 1.51s
701:	learn: 0.3994952	total: 3.55s	remaining: 1.5s
702:	learn: 0.3994512	total: 3.56s	remaining: 1.5s
703:	learn: 0.3994372	total: 3.57s	remaining: 1.5s
704:	learn: 0.3994187	total: 3.57s	remaining: 1.49s
705:	learn: 0.3993986	total: 3.58s	remaining: 1.49s
706:	learn: 0.3993691	total: 3.58s	remaining: 1.48s
707:	learn: 0.3993149	total: 3.59s	remaining: 1.48s
708:	learn: 0.399

873:	learn: 0.3928733	total: 4.31s	remaining: 621ms
874:	learn: 0.3928520	total: 4.31s	remaining: 616ms
875:	learn: 0.3928432	total: 4.32s	remaining: 611ms
876:	learn: 0.3928421	total: 4.32s	remaining: 606ms
877:	learn: 0.3927852	total: 4.33s	remaining: 601ms
878:	learn: 0.3927685	total: 4.33s	remaining: 596ms
879:	learn: 0.3927371	total: 4.34s	remaining: 592ms
880:	learn: 0.3926994	total: 4.34s	remaining: 587ms
881:	learn: 0.3926419	total: 4.35s	remaining: 582ms
882:	learn: 0.3926159	total: 4.35s	remaining: 577ms
883:	learn: 0.3925730	total: 4.36s	remaining: 572ms
884:	learn: 0.3925382	total: 4.36s	remaining: 567ms
885:	learn: 0.3925160	total: 4.37s	remaining: 562ms
886:	learn: 0.3924578	total: 4.37s	remaining: 557ms
887:	learn: 0.3924438	total: 4.37s	remaining: 552ms
888:	learn: 0.3924399	total: 4.38s	remaining: 547ms
889:	learn: 0.3924084	total: 4.39s	remaining: 542ms
890:	learn: 0.3923838	total: 4.39s	remaining: 537ms
891:	learn: 0.3923512	total: 4.4s	remaining: 532ms
892:	learn: 0

In [18]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

preds = (preds >= 0.5).astype(int)
print("F1_score: ", f1_score(y_test, preds))

RMSE: 0.442080
F1_score:  0.6238377007607777


In [21]:
model = CatBoostRegressor(iterations=1250, learning_rate=0.01,
                           depth = 7, random_seed=41)
model.fit(X_train, y_train)
preds = model.predict(X_test)

0:	learn: 0.4943248	total: 5.57ms	remaining: 6.95s
1:	learn: 0.4935052	total: 10.7ms	remaining: 6.71s
2:	learn: 0.4927762	total: 15.9ms	remaining: 6.6s
3:	learn: 0.4919559	total: 21.1ms	remaining: 6.56s
4:	learn: 0.4912490	total: 23.9ms	remaining: 5.96s
5:	learn: 0.4905749	total: 29ms	remaining: 6.01s
6:	learn: 0.4898118	total: 34.5ms	remaining: 6.13s
7:	learn: 0.4890648	total: 39.6ms	remaining: 6.15s
8:	learn: 0.4883679	total: 44.8ms	remaining: 6.18s
9:	learn: 0.4876758	total: 50ms	remaining: 6.21s
10:	learn: 0.4870893	total: 53.2ms	remaining: 6s
11:	learn: 0.4863941	total: 58.6ms	remaining: 6.05s
12:	learn: 0.4857690	total: 63.6ms	remaining: 6.05s
13:	learn: 0.4851549	total: 68.4ms	remaining: 6.04s
14:	learn: 0.4844420	total: 73.4ms	remaining: 6.04s
15:	learn: 0.4838115	total: 78.4ms	remaining: 6.05s
16:	learn: 0.4832128	total: 82.3ms	remaining: 5.97s
17:	learn: 0.4826030	total: 87.4ms	remaining: 5.98s
18:	learn: 0.4820027	total: 92.4ms	remaining: 5.99s
19:	learn: 0.4814333	total: 95

179:	learn: 0.4462751	total: 1.02s	remaining: 6.09s
180:	learn: 0.4462054	total: 1.03s	remaining: 6.08s
181:	learn: 0.4461300	total: 1.04s	remaining: 6.08s
182:	learn: 0.4460355	total: 1.04s	remaining: 6.08s
183:	learn: 0.4459500	total: 1.05s	remaining: 6.08s
184:	learn: 0.4458693	total: 1.05s	remaining: 6.08s
185:	learn: 0.4458317	total: 1.06s	remaining: 6.06s
186:	learn: 0.4457304	total: 1.07s	remaining: 6.06s
187:	learn: 0.4456423	total: 1.07s	remaining: 6.06s
188:	learn: 0.4455729	total: 1.08s	remaining: 6.07s
189:	learn: 0.4455004	total: 1.09s	remaining: 6.08s
190:	learn: 0.4454392	total: 1.1s	remaining: 6.12s
191:	learn: 0.4453692	total: 1.11s	remaining: 6.12s
192:	learn: 0.4453043	total: 1.12s	remaining: 6.13s
193:	learn: 0.4452481	total: 1.13s	remaining: 6.17s
194:	learn: 0.4452045	total: 1.15s	remaining: 6.22s
195:	learn: 0.4451807	total: 1.15s	remaining: 6.2s
196:	learn: 0.4451005	total: 1.16s	remaining: 6.19s
197:	learn: 0.4450081	total: 1.16s	remaining: 6.19s
198:	learn: 0.

339:	learn: 0.4386784	total: 2.04s	remaining: 5.47s
340:	learn: 0.4386508	total: 2.05s	remaining: 5.47s
341:	learn: 0.4386117	total: 2.06s	remaining: 5.46s
342:	learn: 0.4385699	total: 2.06s	remaining: 5.45s
343:	learn: 0.4385457	total: 2.07s	remaining: 5.45s
344:	learn: 0.4385118	total: 2.07s	remaining: 5.44s
345:	learn: 0.4384734	total: 2.08s	remaining: 5.44s
346:	learn: 0.4384280	total: 2.09s	remaining: 5.43s
347:	learn: 0.4383975	total: 2.09s	remaining: 5.43s
348:	learn: 0.4383765	total: 2.1s	remaining: 5.43s
349:	learn: 0.4383454	total: 2.11s	remaining: 5.42s
350:	learn: 0.4382773	total: 2.12s	remaining: 5.42s
351:	learn: 0.4382622	total: 2.12s	remaining: 5.42s
352:	learn: 0.4382272	total: 2.13s	remaining: 5.42s
353:	learn: 0.4382000	total: 2.15s	remaining: 5.43s
354:	learn: 0.4381882	total: 2.15s	remaining: 5.42s
355:	learn: 0.4381512	total: 2.16s	remaining: 5.42s
356:	learn: 0.4381154	total: 2.17s	remaining: 5.42s
357:	learn: 0.4380752	total: 2.17s	remaining: 5.41s
358:	learn: 0

516:	learn: 0.4339364	total: 3.06s	remaining: 4.34s
517:	learn: 0.4339080	total: 3.07s	remaining: 4.33s
518:	learn: 0.4339012	total: 3.07s	remaining: 4.33s
519:	learn: 0.4338910	total: 3.08s	remaining: 4.32s
520:	learn: 0.4338698	total: 3.09s	remaining: 4.32s
521:	learn: 0.4338372	total: 3.09s	remaining: 4.31s
522:	learn: 0.4338190	total: 3.1s	remaining: 4.31s
523:	learn: 0.4337963	total: 3.11s	remaining: 4.31s
524:	learn: 0.4337844	total: 3.11s	remaining: 4.3s
525:	learn: 0.4337574	total: 3.12s	remaining: 4.29s
526:	learn: 0.4337413	total: 3.13s	remaining: 4.29s
527:	learn: 0.4337348	total: 3.13s	remaining: 4.28s
528:	learn: 0.4337337	total: 3.13s	remaining: 4.27s
529:	learn: 0.4336995	total: 3.14s	remaining: 4.26s
530:	learn: 0.4336823	total: 3.14s	remaining: 4.26s
531:	learn: 0.4336766	total: 3.15s	remaining: 4.25s
532:	learn: 0.4336394	total: 3.15s	remaining: 4.24s
533:	learn: 0.4336114	total: 3.16s	remaining: 4.24s
534:	learn: 0.4335935	total: 3.16s	remaining: 4.23s
535:	learn: 0.

679:	learn: 0.4311049	total: 3.9s	remaining: 3.27s
680:	learn: 0.4310841	total: 3.91s	remaining: 3.27s
681:	learn: 0.4310760	total: 3.92s	remaining: 3.26s
682:	learn: 0.4310491	total: 3.92s	remaining: 3.26s
683:	learn: 0.4310155	total: 3.93s	remaining: 3.25s
684:	learn: 0.4309938	total: 3.93s	remaining: 3.25s
685:	learn: 0.4309780	total: 3.94s	remaining: 3.24s
686:	learn: 0.4309373	total: 3.94s	remaining: 3.23s
687:	learn: 0.4309258	total: 3.95s	remaining: 3.23s
688:	learn: 0.4309170	total: 3.96s	remaining: 3.22s
689:	learn: 0.4309060	total: 3.96s	remaining: 3.21s
690:	learn: 0.4308749	total: 3.97s	remaining: 3.21s
691:	learn: 0.4308652	total: 3.97s	remaining: 3.2s
692:	learn: 0.4308419	total: 3.98s	remaining: 3.19s
693:	learn: 0.4308321	total: 3.98s	remaining: 3.19s
694:	learn: 0.4308231	total: 3.98s	remaining: 3.18s
695:	learn: 0.4308094	total: 3.99s	remaining: 3.17s
696:	learn: 0.4307732	total: 4s	remaining: 3.17s
697:	learn: 0.4307420	total: 4s	remaining: 3.16s
698:	learn: 0.430738

838:	learn: 0.4281734	total: 4.75s	remaining: 2.33s
839:	learn: 0.4281529	total: 4.76s	remaining: 2.32s
840:	learn: 0.4281233	total: 4.76s	remaining: 2.31s
841:	learn: 0.4280863	total: 4.78s	remaining: 2.31s
842:	learn: 0.4280501	total: 4.78s	remaining: 2.31s
843:	learn: 0.4280175	total: 4.79s	remaining: 2.3s
844:	learn: 0.4280048	total: 4.79s	remaining: 2.3s
845:	learn: 0.4279911	total: 4.8s	remaining: 2.29s
846:	learn: 0.4279853	total: 4.8s	remaining: 2.28s
847:	learn: 0.4279474	total: 4.81s	remaining: 2.28s
848:	learn: 0.4279362	total: 4.81s	remaining: 2.27s
849:	learn: 0.4278829	total: 4.82s	remaining: 2.27s
850:	learn: 0.4278770	total: 4.82s	remaining: 2.26s
851:	learn: 0.4278642	total: 4.83s	remaining: 2.26s
852:	learn: 0.4278565	total: 4.83s	remaining: 2.25s
853:	learn: 0.4278461	total: 4.84s	remaining: 2.24s
854:	learn: 0.4278244	total: 4.84s	remaining: 2.24s
855:	learn: 0.4278108	total: 4.85s	remaining: 2.23s
856:	learn: 0.4277873	total: 4.86s	remaining: 2.23s
857:	learn: 0.42

1020:	learn: 0.4241030	total: 5.76s	remaining: 1.29s
1021:	learn: 0.4240883	total: 5.77s	remaining: 1.29s
1022:	learn: 0.4240757	total: 5.78s	remaining: 1.28s
1023:	learn: 0.4240488	total: 5.79s	remaining: 1.28s
1024:	learn: 0.4240371	total: 5.8s	remaining: 1.27s
1025:	learn: 0.4240248	total: 5.8s	remaining: 1.27s
1026:	learn: 0.4239943	total: 5.81s	remaining: 1.26s
1027:	learn: 0.4239777	total: 5.82s	remaining: 1.26s
1028:	learn: 0.4239619	total: 5.82s	remaining: 1.25s
1029:	learn: 0.4239515	total: 5.83s	remaining: 1.24s
1030:	learn: 0.4239241	total: 5.83s	remaining: 1.24s
1031:	learn: 0.4239098	total: 5.84s	remaining: 1.23s
1032:	learn: 0.4238958	total: 5.84s	remaining: 1.23s
1033:	learn: 0.4238479	total: 5.85s	remaining: 1.22s
1034:	learn: 0.4238246	total: 5.85s	remaining: 1.22s
1035:	learn: 0.4238060	total: 5.86s	remaining: 1.21s
1036:	learn: 0.4237831	total: 5.86s	remaining: 1.2s
1037:	learn: 0.4237720	total: 5.87s	remaining: 1.2s
1038:	learn: 0.4237457	total: 5.87s	remaining: 1.1

1206:	learn: 0.4202307	total: 6.79s	remaining: 242ms
1207:	learn: 0.4202110	total: 6.8s	remaining: 236ms
1208:	learn: 0.4201647	total: 6.81s	remaining: 231ms
1209:	learn: 0.4201466	total: 6.81s	remaining: 225ms
1210:	learn: 0.4201321	total: 6.82s	remaining: 220ms
1211:	learn: 0.4201056	total: 6.83s	remaining: 214ms
1212:	learn: 0.4200948	total: 6.83s	remaining: 208ms
1213:	learn: 0.4200868	total: 6.83s	remaining: 203ms
1214:	learn: 0.4200593	total: 6.84s	remaining: 197ms
1215:	learn: 0.4200349	total: 6.85s	remaining: 191ms
1216:	learn: 0.4200112	total: 6.85s	remaining: 186ms
1217:	learn: 0.4199977	total: 6.86s	remaining: 180ms
1218:	learn: 0.4199746	total: 6.86s	remaining: 174ms
1219:	learn: 0.4199598	total: 6.87s	remaining: 169ms
1220:	learn: 0.4199451	total: 6.87s	remaining: 163ms
1221:	learn: 0.4199364	total: 6.88s	remaining: 158ms
1222:	learn: 0.4199139	total: 6.88s	remaining: 152ms
1223:	learn: 0.4198826	total: 6.89s	remaining: 146ms
1224:	learn: 0.4198616	total: 6.89s	remaining: 

In [22]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

preds = (preds >= 0.5).astype(int)
print("F1_score: ", f1_score(y_test, preds))

RMSE: 0.439847
F1_score:  0.6206308610400681


In [23]:
#Pruebo con CatBoostClassifier
#CatboostClassifier por defecto
model = CatBoostClassifier(random_seed = 41)
model.fit(X_train, y_train)
preds = model.predict(X_test)

Learning rate set to 0.022283
0:	learn: 0.6881339	total: 7.23ms	remaining: 7.22s
1:	learn: 0.6834910	total: 14.1ms	remaining: 7.05s
2:	learn: 0.6801623	total: 18.7ms	remaining: 6.21s
3:	learn: 0.6764096	total: 25.6ms	remaining: 6.37s
4:	learn: 0.6731570	total: 33.2ms	remaining: 6.6s
5:	learn: 0.6691407	total: 40.2ms	remaining: 6.66s
6:	learn: 0.6649513	total: 47.3ms	remaining: 6.71s
7:	learn: 0.6611457	total: 54.3ms	remaining: 6.73s
8:	learn: 0.6581740	total: 61.1ms	remaining: 6.73s
9:	learn: 0.6553907	total: 67.8ms	remaining: 6.71s
10:	learn: 0.6525744	total: 75ms	remaining: 6.75s
11:	learn: 0.6498927	total: 82.1ms	remaining: 6.76s
12:	learn: 0.6468371	total: 89.5ms	remaining: 6.79s
13:	learn: 0.6442128	total: 96.5ms	remaining: 6.79s
14:	learn: 0.6414336	total: 104ms	remaining: 6.82s
15:	learn: 0.6385736	total: 111ms	remaining: 6.82s
16:	learn: 0.6364580	total: 118ms	remaining: 6.81s
17:	learn: 0.6348948	total: 122ms	remaining: 6.68s
18:	learn: 0.6325626	total: 129ms	remaining: 6.69s


170:	learn: 0.5634857	total: 1.42s	remaining: 6.89s
171:	learn: 0.5634431	total: 1.43s	remaining: 6.88s
172:	learn: 0.5633482	total: 1.44s	remaining: 6.9s
173:	learn: 0.5632581	total: 1.45s	remaining: 6.89s
174:	learn: 0.5630650	total: 1.46s	remaining: 6.87s
175:	learn: 0.5629522	total: 1.47s	remaining: 6.86s
176:	learn: 0.5629389	total: 1.47s	remaining: 6.84s
177:	learn: 0.5627990	total: 1.48s	remaining: 6.82s
178:	learn: 0.5626435	total: 1.48s	remaining: 6.81s
179:	learn: 0.5625254	total: 1.49s	remaining: 6.79s
180:	learn: 0.5624150	total: 1.5s	remaining: 6.78s
181:	learn: 0.5623352	total: 1.51s	remaining: 6.77s
182:	learn: 0.5623251	total: 1.51s	remaining: 6.75s
183:	learn: 0.5622195	total: 1.52s	remaining: 6.73s
184:	learn: 0.5620687	total: 1.52s	remaining: 6.72s
185:	learn: 0.5619428	total: 1.53s	remaining: 6.71s
186:	learn: 0.5619063	total: 1.54s	remaining: 6.7s
187:	learn: 0.5618226	total: 1.55s	remaining: 6.68s
188:	learn: 0.5616596	total: 1.55s	remaining: 6.67s
189:	learn: 0.5

344:	learn: 0.5469547	total: 2.81s	remaining: 5.34s
345:	learn: 0.5469246	total: 2.82s	remaining: 5.33s
346:	learn: 0.5468397	total: 2.83s	remaining: 5.32s
347:	learn: 0.5468176	total: 2.84s	remaining: 5.32s
348:	learn: 0.5466689	total: 2.85s	remaining: 5.31s
349:	learn: 0.5465839	total: 2.85s	remaining: 5.3s
350:	learn: 0.5464869	total: 2.86s	remaining: 5.29s
351:	learn: 0.5464375	total: 2.87s	remaining: 5.28s
352:	learn: 0.5462495	total: 2.88s	remaining: 5.27s
353:	learn: 0.5460864	total: 2.88s	remaining: 5.26s
354:	learn: 0.5460625	total: 2.89s	remaining: 5.25s
355:	learn: 0.5459567	total: 2.9s	remaining: 5.25s
356:	learn: 0.5459216	total: 2.91s	remaining: 5.24s
357:	learn: 0.5458847	total: 2.91s	remaining: 5.22s
358:	learn: 0.5456888	total: 2.92s	remaining: 5.21s
359:	learn: 0.5455082	total: 2.93s	remaining: 5.2s
360:	learn: 0.5453953	total: 2.93s	remaining: 5.19s
361:	learn: 0.5453810	total: 2.94s	remaining: 5.18s
362:	learn: 0.5452671	total: 2.95s	remaining: 5.18s
363:	learn: 0.5

520:	learn: 0.5281174	total: 4.21s	remaining: 3.87s
521:	learn: 0.5280568	total: 4.22s	remaining: 3.87s
522:	learn: 0.5279885	total: 4.23s	remaining: 3.86s
523:	learn: 0.5278543	total: 4.24s	remaining: 3.85s
524:	learn: 0.5277332	total: 4.24s	remaining: 3.84s
525:	learn: 0.5276662	total: 4.25s	remaining: 3.83s
526:	learn: 0.5275206	total: 4.26s	remaining: 3.82s
527:	learn: 0.5274631	total: 4.26s	remaining: 3.81s
528:	learn: 0.5273855	total: 4.27s	remaining: 3.8s
529:	learn: 0.5272739	total: 4.28s	remaining: 3.79s
530:	learn: 0.5271160	total: 4.29s	remaining: 3.79s
531:	learn: 0.5269841	total: 4.29s	remaining: 3.78s
532:	learn: 0.5269179	total: 4.3s	remaining: 3.77s
533:	learn: 0.5268572	total: 4.31s	remaining: 3.76s
534:	learn: 0.5267936	total: 4.31s	remaining: 3.75s
535:	learn: 0.5267490	total: 4.32s	remaining: 3.74s
536:	learn: 0.5266866	total: 4.33s	remaining: 3.73s
537:	learn: 0.5266071	total: 4.33s	remaining: 3.72s
538:	learn: 0.5264727	total: 4.34s	remaining: 3.71s
539:	learn: 0.

683:	learn: 0.5140705	total: 5.44s	remaining: 2.51s
684:	learn: 0.5139770	total: 5.45s	remaining: 2.5s
685:	learn: 0.5139120	total: 5.46s	remaining: 2.5s
686:	learn: 0.5137979	total: 5.47s	remaining: 2.49s
687:	learn: 0.5137168	total: 5.48s	remaining: 2.48s
688:	learn: 0.5135773	total: 5.49s	remaining: 2.48s
689:	learn: 0.5134671	total: 5.5s	remaining: 2.47s
690:	learn: 0.5134012	total: 5.51s	remaining: 2.46s
691:	learn: 0.5133644	total: 5.51s	remaining: 2.45s
692:	learn: 0.5132651	total: 5.52s	remaining: 2.44s
693:	learn: 0.5132092	total: 5.53s	remaining: 2.44s
694:	learn: 0.5131051	total: 5.53s	remaining: 2.43s
695:	learn: 0.5130579	total: 5.54s	remaining: 2.42s
696:	learn: 0.5129235	total: 5.55s	remaining: 2.41s
697:	learn: 0.5127950	total: 5.56s	remaining: 2.4s
698:	learn: 0.5126680	total: 5.56s	remaining: 2.4s
699:	learn: 0.5126175	total: 5.57s	remaining: 2.39s
700:	learn: 0.5125165	total: 5.58s	remaining: 2.38s
701:	learn: 0.5124897	total: 5.59s	remaining: 2.37s
702:	learn: 0.512

861:	learn: 0.5004659	total: 6.84s	remaining: 1.09s
862:	learn: 0.5003716	total: 6.85s	remaining: 1.09s
863:	learn: 0.5003121	total: 6.86s	remaining: 1.08s
864:	learn: 0.5002637	total: 6.87s	remaining: 1.07s
865:	learn: 0.5001435	total: 6.88s	remaining: 1.06s
866:	learn: 0.5001253	total: 6.89s	remaining: 1.06s
867:	learn: 0.4999976	total: 6.9s	remaining: 1.05s
868:	learn: 0.4999080	total: 6.91s	remaining: 1.04s
869:	learn: 0.4998473	total: 6.92s	remaining: 1.03s
870:	learn: 0.4998211	total: 6.93s	remaining: 1.02s
871:	learn: 0.4997050	total: 6.93s	remaining: 1.02s
872:	learn: 0.4996308	total: 6.94s	remaining: 1.01s
873:	learn: 0.4995683	total: 6.95s	remaining: 1s
874:	learn: 0.4995006	total: 6.96s	remaining: 994ms
875:	learn: 0.4994051	total: 6.97s	remaining: 986ms
876:	learn: 0.4993305	total: 6.97s	remaining: 978ms
877:	learn: 0.4992557	total: 6.98s	remaining: 970ms
878:	learn: 0.4992006	total: 6.99s	remaining: 962ms
879:	learn: 0.4991746	total: 7s	remaining: 954ms
880:	learn: 0.49904

In [24]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

print("F1_score: ", f1_score(y_test, preds))

RMSE: 0.543571
F1_score:  0.6173469387755102


In [25]:
model = CatBoostClassifier(iterations = 1250, learning_rate = 0.01,
                           depth = 7, random_seed = 41)
model.fit(X_train, y_train)
preds = model.predict(X_test)

0:	learn: 0.6908385	total: 8.39ms	remaining: 10.5s
1:	learn: 0.6888662	total: 16.4ms	remaining: 10.3s
2:	learn: 0.6866983	total: 24.6ms	remaining: 10.2s
3:	learn: 0.6848295	total: 32.2ms	remaining: 10s
4:	learn: 0.6832139	total: 37.2ms	remaining: 9.26s
5:	learn: 0.6811713	total: 45.3ms	remaining: 9.38s
6:	learn: 0.6793942	total: 53.5ms	remaining: 9.51s
7:	learn: 0.6771978	total: 62ms	remaining: 9.63s
8:	learn: 0.6751186	total: 70.1ms	remaining: 9.67s
9:	learn: 0.6735083	total: 77.1ms	remaining: 9.56s
10:	learn: 0.6719591	total: 85.4ms	remaining: 9.61s
11:	learn: 0.6699953	total: 93.5ms	remaining: 9.65s
12:	learn: 0.6683183	total: 99.8ms	remaining: 9.49s
13:	learn: 0.6665101	total: 108ms	remaining: 9.53s
14:	learn: 0.6645567	total: 116ms	remaining: 9.56s
15:	learn: 0.6630689	total: 124ms	remaining: 9.58s
16:	learn: 0.6616089	total: 133ms	remaining: 9.64s
17:	learn: 0.6598867	total: 142ms	remaining: 9.69s
18:	learn: 0.6584803	total: 151ms	remaining: 9.81s
19:	learn: 0.6570482	total: 160m

178:	learn: 0.5769262	total: 1.58s	remaining: 9.46s
179:	learn: 0.5767559	total: 1.6s	remaining: 9.5s
180:	learn: 0.5766298	total: 1.62s	remaining: 9.54s
181:	learn: 0.5764332	total: 1.63s	remaining: 9.54s
182:	learn: 0.5763153	total: 1.63s	remaining: 9.53s
183:	learn: 0.5761546	total: 1.64s	remaining: 9.52s
184:	learn: 0.5759412	total: 1.65s	remaining: 9.51s
185:	learn: 0.5757053	total: 1.66s	remaining: 9.5s
186:	learn: 0.5755655	total: 1.67s	remaining: 9.49s
187:	learn: 0.5753316	total: 1.68s	remaining: 9.48s
188:	learn: 0.5751852	total: 1.69s	remaining: 9.46s
189:	learn: 0.5750238	total: 1.69s	remaining: 9.45s
190:	learn: 0.5748807	total: 1.7s	remaining: 9.44s
191:	learn: 0.5746431	total: 1.71s	remaining: 9.42s
192:	learn: 0.5744965	total: 1.72s	remaining: 9.41s
193:	learn: 0.5744283	total: 1.72s	remaining: 9.38s
194:	learn: 0.5742585	total: 1.73s	remaining: 9.37s
195:	learn: 0.5741167	total: 1.74s	remaining: 9.36s
196:	learn: 0.5739761	total: 1.75s	remaining: 9.35s
197:	learn: 0.57

337:	learn: 0.5591333	total: 2.99s	remaining: 8.07s
338:	learn: 0.5590529	total: 3s	remaining: 8.06s
339:	learn: 0.5589065	total: 3.01s	remaining: 8.06s
340:	learn: 0.5588562	total: 3.02s	remaining: 8.05s
341:	learn: 0.5587463	total: 3.03s	remaining: 8.05s
342:	learn: 0.5586797	total: 3.04s	remaining: 8.04s
343:	learn: 0.5586255	total: 3.05s	remaining: 8.04s
344:	learn: 0.5585566	total: 3.06s	remaining: 8.03s
345:	learn: 0.5585044	total: 3.07s	remaining: 8.03s
346:	learn: 0.5584354	total: 3.08s	remaining: 8.02s
347:	learn: 0.5582485	total: 3.09s	remaining: 8.02s
348:	learn: 0.5582011	total: 3.1s	remaining: 8.01s
349:	learn: 0.5580888	total: 3.11s	remaining: 8.01s
350:	learn: 0.5580295	total: 3.12s	remaining: 8s
351:	learn: 0.5579240	total: 3.13s	remaining: 8s
352:	learn: 0.5577856	total: 3.15s	remaining: 8s
353:	learn: 0.5577285	total: 3.16s	remaining: 7.99s
354:	learn: 0.5576409	total: 3.17s	remaining: 7.99s
355:	learn: 0.5575673	total: 3.18s	remaining: 7.99s
356:	learn: 0.5574673	tot

512:	learn: 0.5478194	total: 4.59s	remaining: 6.6s
513:	learn: 0.5477533	total: 4.61s	remaining: 6.59s
514:	learn: 0.5476977	total: 4.62s	remaining: 6.6s
515:	learn: 0.5476480	total: 4.63s	remaining: 6.59s
516:	learn: 0.5475749	total: 4.64s	remaining: 6.58s
517:	learn: 0.5475433	total: 4.65s	remaining: 6.57s
518:	learn: 0.5475311	total: 4.66s	remaining: 6.56s
519:	learn: 0.5475139	total: 4.66s	remaining: 6.55s
520:	learn: 0.5474146	total: 4.67s	remaining: 6.54s
521:	learn: 0.5473041	total: 4.68s	remaining: 6.53s
522:	learn: 0.5472927	total: 4.69s	remaining: 6.51s
523:	learn: 0.5472417	total: 4.7s	remaining: 6.5s
524:	learn: 0.5471908	total: 4.7s	remaining: 6.5s
525:	learn: 0.5471460	total: 4.71s	remaining: 6.49s
526:	learn: 0.5471081	total: 4.72s	remaining: 6.48s
527:	learn: 0.5470609	total: 4.73s	remaining: 6.47s
528:	learn: 0.5470253	total: 4.74s	remaining: 6.46s
529:	learn: 0.5469607	total: 4.75s	remaining: 6.45s
530:	learn: 0.5469288	total: 4.75s	remaining: 6.44s
531:	learn: 0.5468

678:	learn: 0.5402256	total: 6s	remaining: 5.05s
679:	learn: 0.5401547	total: 6.01s	remaining: 5.04s
680:	learn: 0.5401017	total: 6.03s	remaining: 5.04s
681:	learn: 0.5400367	total: 6.04s	remaining: 5.03s
682:	learn: 0.5399909	total: 6.04s	remaining: 5.02s
683:	learn: 0.5399490	total: 6.05s	remaining: 5.01s
684:	learn: 0.5398687	total: 6.06s	remaining: 5s
685:	learn: 0.5398187	total: 6.07s	remaining: 4.99s
686:	learn: 0.5397980	total: 6.08s	remaining: 4.98s
687:	learn: 0.5397696	total: 6.08s	remaining: 4.97s
688:	learn: 0.5396987	total: 6.09s	remaining: 4.96s
689:	learn: 0.5396290	total: 6.1s	remaining: 4.95s
690:	learn: 0.5395890	total: 6.11s	remaining: 4.94s
691:	learn: 0.5395772	total: 6.12s	remaining: 4.93s
692:	learn: 0.5395739	total: 6.13s	remaining: 4.92s
693:	learn: 0.5395182	total: 6.13s	remaining: 4.91s
694:	learn: 0.5394949	total: 6.14s	remaining: 4.91s
695:	learn: 0.5394556	total: 6.15s	remaining: 4.9s
696:	learn: 0.5394550	total: 6.16s	remaining: 4.88s
697:	learn: 0.539410

846:	learn: 0.5320066	total: 7.61s	remaining: 3.62s
847:	learn: 0.5319623	total: 7.62s	remaining: 3.61s
848:	learn: 0.5319206	total: 7.63s	remaining: 3.6s
849:	learn: 0.5318712	total: 7.64s	remaining: 3.6s
850:	learn: 0.5318282	total: 7.66s	remaining: 3.59s
851:	learn: 0.5317645	total: 7.67s	remaining: 3.58s
852:	learn: 0.5316508	total: 7.68s	remaining: 3.57s
853:	learn: 0.5315912	total: 7.7s	remaining: 3.57s
854:	learn: 0.5315387	total: 7.71s	remaining: 3.56s
855:	learn: 0.5314962	total: 7.72s	remaining: 3.55s
856:	learn: 0.5314421	total: 7.72s	remaining: 3.54s
857:	learn: 0.5313803	total: 7.73s	remaining: 3.53s
858:	learn: 0.5313341	total: 7.74s	remaining: 3.52s
859:	learn: 0.5312929	total: 7.75s	remaining: 3.51s
860:	learn: 0.5312469	total: 7.76s	remaining: 3.5s
861:	learn: 0.5311901	total: 7.76s	remaining: 3.5s
862:	learn: 0.5311548	total: 7.77s	remaining: 3.48s
863:	learn: 0.5310533	total: 7.78s	remaining: 3.48s
864:	learn: 0.5310232	total: 7.79s	remaining: 3.47s
865:	learn: 0.530

1018:	learn: 0.5214990	total: 9.19s	remaining: 2.08s
1019:	learn: 0.5213983	total: 9.2s	remaining: 2.07s
1020:	learn: 0.5213269	total: 9.21s	remaining: 2.07s
1021:	learn: 0.5212852	total: 9.22s	remaining: 2.06s
1022:	learn: 0.5211973	total: 9.23s	remaining: 2.05s
1023:	learn: 0.5211778	total: 9.24s	remaining: 2.04s
1024:	learn: 0.5211254	total: 9.24s	remaining: 2.03s
1025:	learn: 0.5210478	total: 9.25s	remaining: 2.02s
1026:	learn: 0.5210155	total: 9.26s	remaining: 2.01s
1027:	learn: 0.5209657	total: 9.27s	remaining: 2s
1028:	learn: 0.5208943	total: 9.28s	remaining: 1.99s
1029:	learn: 0.5208026	total: 9.29s	remaining: 1.98s
1030:	learn: 0.5207677	total: 9.29s	remaining: 1.97s
1031:	learn: 0.5207073	total: 9.3s	remaining: 1.97s
1032:	learn: 0.5206582	total: 9.31s	remaining: 1.96s
1033:	learn: 0.5205900	total: 9.32s	remaining: 1.95s
1034:	learn: 0.5205189	total: 9.33s	remaining: 1.94s
1035:	learn: 0.5204520	total: 9.34s	remaining: 1.93s
1036:	learn: 0.5203640	total: 9.34s	remaining: 1.92

1188:	learn: 0.5117167	total: 10.8s	remaining: 553ms
1189:	learn: 0.5116779	total: 10.8s	remaining: 544ms
1190:	learn: 0.5116336	total: 10.8s	remaining: 535ms
1191:	learn: 0.5115941	total: 10.8s	remaining: 526ms
1192:	learn: 0.5115262	total: 10.8s	remaining: 517ms
1193:	learn: 0.5114854	total: 10.8s	remaining: 508ms
1194:	learn: 0.5114374	total: 10.8s	remaining: 499ms
1195:	learn: 0.5113832	total: 10.8s	remaining: 490ms
1196:	learn: 0.5113391	total: 10.9s	remaining: 481ms
1197:	learn: 0.5113178	total: 10.9s	remaining: 472ms
1198:	learn: 0.5112768	total: 10.9s	remaining: 463ms
1199:	learn: 0.5112036	total: 10.9s	remaining: 454ms
1200:	learn: 0.5111365	total: 10.9s	remaining: 444ms
1201:	learn: 0.5110835	total: 10.9s	remaining: 435ms
1202:	learn: 0.5110215	total: 10.9s	remaining: 426ms
1203:	learn: 0.5109745	total: 10.9s	remaining: 417ms
1204:	learn: 0.5109246	total: 10.9s	remaining: 408ms
1205:	learn: 0.5108977	total: 10.9s	remaining: 399ms
1206:	learn: 0.5108426	total: 11s	remaining: 3

In [26]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

print("F1_score: ", f1_score(y_test, preds))

RMSE: 0.534435
F1_score:  0.6285226302305722


**Agregando los features categóricos de keyword y location**

In [42]:
#construyendo los datos de entrenamiento y de test
selected_df = train[['keyword','location','length','amount_words','amount_keywords_in_tweet','amount_locations_in_tweet', 'amount_urls','amount_users', 'amount_hashtags','len_hash_over_text', 'has_url']] 

X, y = selected_df,train['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 41)

In [43]:
#Pruebo primero con CatBoostRegressor
#CatBoostRegressor por defecto
model = CatBoostRegressor(random_seed = 41, one_hot_max_size = 3500)
model.fit(X_train, y_train, cat_features = [0,1])
preds = model.predict(X_test)

Learning rate set to 0.053835
0:	learn: 0.4910489	total: 27.4ms	remaining: 27.3s
1:	learn: 0.4872543	total: 44ms	remaining: 21.9s
2:	learn: 0.4837809	total: 58.5ms	remaining: 19.5s
3:	learn: 0.4804971	total: 73.1ms	remaining: 18.2s
4:	learn: 0.4775547	total: 87.5ms	remaining: 17.4s
5:	learn: 0.4751884	total: 122ms	remaining: 20.2s
6:	learn: 0.4729827	total: 138ms	remaining: 19.6s
7:	learn: 0.4708908	total: 155ms	remaining: 19.2s
8:	learn: 0.4688714	total: 170ms	remaining: 18.7s
9:	learn: 0.4672581	total: 184ms	remaining: 18.3s
10:	learn: 0.4657951	total: 191ms	remaining: 17.2s
11:	learn: 0.4642174	total: 205ms	remaining: 16.9s
12:	learn: 0.4625907	total: 224ms	remaining: 17s
13:	learn: 0.4611964	total: 241ms	remaining: 17s
14:	learn: 0.4600364	total: 256ms	remaining: 16.8s
15:	learn: 0.4590118	total: 271ms	remaining: 16.7s
16:	learn: 0.4581311	total: 285ms	remaining: 16.5s
17:	learn: 0.4571319	total: 301ms	remaining: 16.4s
18:	learn: 0.4562445	total: 317ms	remaining: 16.4s
19:	learn: 0

166:	learn: 0.4237350	total: 2.77s	remaining: 13.8s
167:	learn: 0.4236041	total: 2.78s	remaining: 13.8s
168:	learn: 0.4234331	total: 2.8s	remaining: 13.8s
169:	learn: 0.4232849	total: 2.82s	remaining: 13.8s
170:	learn: 0.4231293	total: 2.83s	remaining: 13.7s
171:	learn: 0.4229834	total: 2.85s	remaining: 13.7s
172:	learn: 0.4228116	total: 2.86s	remaining: 13.7s
173:	learn: 0.4226248	total: 2.87s	remaining: 13.6s
174:	learn: 0.4224806	total: 2.89s	remaining: 13.6s
175:	learn: 0.4223437	total: 2.9s	remaining: 13.6s
176:	learn: 0.4221534	total: 2.92s	remaining: 13.6s
177:	learn: 0.4219522	total: 2.93s	remaining: 13.6s
178:	learn: 0.4217948	total: 2.95s	remaining: 13.5s
179:	learn: 0.4215888	total: 2.98s	remaining: 13.6s
180:	learn: 0.4213926	total: 2.99s	remaining: 13.5s
181:	learn: 0.4212082	total: 3.01s	remaining: 13.5s
182:	learn: 0.4210699	total: 3.02s	remaining: 13.5s
183:	learn: 0.4208907	total: 3.04s	remaining: 13.5s
184:	learn: 0.4207359	total: 3.05s	remaining: 13.4s
185:	learn: 0.

328:	learn: 0.3998308	total: 5.34s	remaining: 10.9s
329:	learn: 0.3996510	total: 5.36s	remaining: 10.9s
330:	learn: 0.3995260	total: 5.37s	remaining: 10.9s
331:	learn: 0.3994201	total: 5.38s	remaining: 10.8s
332:	learn: 0.3992848	total: 5.4s	remaining: 10.8s
333:	learn: 0.3991748	total: 5.41s	remaining: 10.8s
334:	learn: 0.3990302	total: 5.43s	remaining: 10.8s
335:	learn: 0.3989143	total: 5.44s	remaining: 10.8s
336:	learn: 0.3988179	total: 5.45s	remaining: 10.7s
337:	learn: 0.3986776	total: 5.47s	remaining: 10.7s
338:	learn: 0.3985339	total: 5.48s	remaining: 10.7s
339:	learn: 0.3984222	total: 5.5s	remaining: 10.7s
340:	learn: 0.3983237	total: 5.52s	remaining: 10.7s
341:	learn: 0.3982154	total: 5.53s	remaining: 10.6s
342:	learn: 0.3981108	total: 5.55s	remaining: 10.6s
343:	learn: 0.3979950	total: 5.57s	remaining: 10.6s
344:	learn: 0.3978915	total: 5.58s	remaining: 10.6s
345:	learn: 0.3977860	total: 5.59s	remaining: 10.6s
346:	learn: 0.3976685	total: 5.61s	remaining: 10.6s
347:	learn: 0.

487:	learn: 0.3835363	total: 7.7s	remaining: 8.08s
488:	learn: 0.3834543	total: 7.71s	remaining: 8.06s
489:	learn: 0.3833781	total: 7.73s	remaining: 8.05s
490:	learn: 0.3832784	total: 7.75s	remaining: 8.03s
491:	learn: 0.3831875	total: 7.76s	remaining: 8.01s
492:	learn: 0.3830819	total: 7.78s	remaining: 8s
493:	learn: 0.3830609	total: 7.79s	remaining: 7.98s
494:	learn: 0.3830301	total: 7.8s	remaining: 7.96s
495:	learn: 0.3829371	total: 7.82s	remaining: 7.94s
496:	learn: 0.3828350	total: 7.83s	remaining: 7.92s
497:	learn: 0.3827610	total: 7.84s	remaining: 7.91s
498:	learn: 0.3826542	total: 7.86s	remaining: 7.89s
499:	learn: 0.3826337	total: 7.87s	remaining: 7.87s
500:	learn: 0.3825515	total: 7.89s	remaining: 7.86s
501:	learn: 0.3824599	total: 7.9s	remaining: 7.84s
502:	learn: 0.3823861	total: 7.92s	remaining: 7.82s
503:	learn: 0.3822838	total: 7.93s	remaining: 7.81s
504:	learn: 0.3821892	total: 7.95s	remaining: 7.79s
505:	learn: 0.3821058	total: 7.96s	remaining: 7.77s
506:	learn: 0.3820

647:	learn: 0.3720459	total: 10.1s	remaining: 5.47s
648:	learn: 0.3720287	total: 10.1s	remaining: 5.45s
649:	learn: 0.3719543	total: 10.1s	remaining: 5.44s
650:	learn: 0.3719223	total: 10.1s	remaining: 5.42s
651:	learn: 0.3718529	total: 10.1s	remaining: 5.41s
652:	learn: 0.3717728	total: 10.1s	remaining: 5.39s
653:	learn: 0.3716783	total: 10.2s	remaining: 5.37s
654:	learn: 0.3715911	total: 10.2s	remaining: 5.36s
655:	learn: 0.3714809	total: 10.2s	remaining: 5.34s
656:	learn: 0.3714406	total: 10.2s	remaining: 5.32s
657:	learn: 0.3714190	total: 10.2s	remaining: 5.31s
658:	learn: 0.3713572	total: 10.2s	remaining: 5.29s
659:	learn: 0.3712893	total: 10.2s	remaining: 5.28s
660:	learn: 0.3712184	total: 10.3s	remaining: 5.26s
661:	learn: 0.3711872	total: 10.3s	remaining: 5.24s
662:	learn: 0.3710832	total: 10.3s	remaining: 5.23s
663:	learn: 0.3710219	total: 10.3s	remaining: 5.21s
664:	learn: 0.3710059	total: 10.3s	remaining: 5.2s
665:	learn: 0.3709725	total: 10.3s	remaining: 5.18s
666:	learn: 0

817:	learn: 0.3626524	total: 12.6s	remaining: 2.8s
818:	learn: 0.3625943	total: 12.6s	remaining: 2.78s
819:	learn: 0.3625224	total: 12.6s	remaining: 2.77s
820:	learn: 0.3624646	total: 12.6s	remaining: 2.75s
821:	learn: 0.3624454	total: 12.6s	remaining: 2.74s
822:	learn: 0.3623929	total: 12.7s	remaining: 2.72s
823:	learn: 0.3623787	total: 12.7s	remaining: 2.71s
824:	learn: 0.3623636	total: 12.7s	remaining: 2.69s
825:	learn: 0.3623322	total: 12.7s	remaining: 2.67s
826:	learn: 0.3622454	total: 12.7s	remaining: 2.66s
827:	learn: 0.3621732	total: 12.7s	remaining: 2.64s
828:	learn: 0.3621579	total: 12.7s	remaining: 2.63s
829:	learn: 0.3620868	total: 12.8s	remaining: 2.61s
830:	learn: 0.3620386	total: 12.8s	remaining: 2.6s
831:	learn: 0.3620226	total: 12.8s	remaining: 2.58s
832:	learn: 0.3619527	total: 12.8s	remaining: 2.57s
833:	learn: 0.3619333	total: 12.8s	remaining: 2.55s
834:	learn: 0.3618812	total: 12.8s	remaining: 2.54s
835:	learn: 0.3618619	total: 12.8s	remaining: 2.52s
836:	learn: 0.

978:	learn: 0.3549431	total: 14.9s	remaining: 321ms
979:	learn: 0.3549198	total: 15s	remaining: 305ms
980:	learn: 0.3548866	total: 15s	remaining: 290ms
981:	learn: 0.3548687	total: 15s	remaining: 275ms
982:	learn: 0.3548266	total: 15s	remaining: 259ms
983:	learn: 0.3547226	total: 15s	remaining: 244ms
984:	learn: 0.3547057	total: 15s	remaining: 229ms
985:	learn: 0.3546881	total: 15s	remaining: 214ms
986:	learn: 0.3546703	total: 15.1s	remaining: 198ms
987:	learn: 0.3545825	total: 15.1s	remaining: 183ms
988:	learn: 0.3545571	total: 15.1s	remaining: 168ms
989:	learn: 0.3544923	total: 15.1s	remaining: 153ms
990:	learn: 0.3544705	total: 15.1s	remaining: 137ms
991:	learn: 0.3543945	total: 15.1s	remaining: 122ms
992:	learn: 0.3543800	total: 15.1s	remaining: 107ms
993:	learn: 0.3542973	total: 15.2s	remaining: 91.5ms
994:	learn: 0.3542814	total: 15.2s	remaining: 76.2ms
995:	learn: 0.3542016	total: 15.2s	remaining: 61ms
996:	learn: 0.3541120	total: 15.2s	remaining: 45.7ms
997:	learn: 0.3540835	to

In [44]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

preds = (preds >= 0.5).astype(int)
print("F1_score: ", f1_score(y_test, preds))

RMSE: 0.399744
F1_score:  0.7230514096185737


In [45]:
model = CatBoostRegressor(random_seed = 41, one_hot_max_size = 3500, iterations=1250, learning_rate=0.01, depth = 7)
model.fit(X_train, y_train, cat_features = [0,1])
preds = model.predict(X_test)

0:	learn: 0.4942435	total: 6.9ms	remaining: 8.62s
1:	learn: 0.4934683	total: 22.5ms	remaining: 14.1s
2:	learn: 0.4926937	total: 48.3ms	remaining: 20.1s
3:	learn: 0.4919712	total: 58.2ms	remaining: 18.1s
4:	learn: 0.4912072	total: 82.5ms	remaining: 20.6s
5:	learn: 0.4904869	total: 108ms	remaining: 22.4s
6:	learn: 0.4897691	total: 132ms	remaining: 23.4s
7:	learn: 0.4891390	total: 141ms	remaining: 21.8s
8:	learn: 0.4884384	total: 165ms	remaining: 22.8s
9:	learn: 0.4877466	total: 190ms	remaining: 23.5s
10:	learn: 0.4871147	total: 219ms	remaining: 24.7s
11:	learn: 0.4864157	total: 246ms	remaining: 25.3s
12:	learn: 0.4857825	total: 273ms	remaining: 26s
13:	learn: 0.4851535	total: 300ms	remaining: 26.5s
14:	learn: 0.4844802	total: 327ms	remaining: 26.9s
15:	learn: 0.4838785	total: 353ms	remaining: 27.3s
16:	learn: 0.4832303	total: 378ms	remaining: 27.4s
17:	learn: 0.4826137	total: 406ms	remaining: 27.8s
18:	learn: 0.4820898	total: 431ms	remaining: 27.9s
19:	learn: 0.4815754	total: 456ms	remai

163:	learn: 0.4476074	total: 4.26s	remaining: 28.2s
164:	learn: 0.4475284	total: 4.29s	remaining: 28.2s
165:	learn: 0.4474178	total: 4.32s	remaining: 28.2s
166:	learn: 0.4473167	total: 4.34s	remaining: 28.2s
167:	learn: 0.4472112	total: 4.37s	remaining: 28.1s
168:	learn: 0.4470940	total: 4.39s	remaining: 28.1s
169:	learn: 0.4470261	total: 4.42s	remaining: 28.1s
170:	learn: 0.4469473	total: 4.45s	remaining: 28.1s
171:	learn: 0.4468577	total: 4.48s	remaining: 28.1s
172:	learn: 0.4467670	total: 4.51s	remaining: 28.1s
173:	learn: 0.4466715	total: 4.53s	remaining: 28s
174:	learn: 0.4465739	total: 4.56s	remaining: 28s
175:	learn: 0.4464735	total: 4.58s	remaining: 28s
176:	learn: 0.4464016	total: 4.61s	remaining: 27.9s
177:	learn: 0.4463374	total: 4.63s	remaining: 27.9s
178:	learn: 0.4462592	total: 4.66s	remaining: 27.9s
179:	learn: 0.4461804	total: 4.69s	remaining: 27.9s
180:	learn: 0.4461006	total: 4.71s	remaining: 27.8s
181:	learn: 0.4460028	total: 4.74s	remaining: 27.8s
182:	learn: 0.4459

324:	learn: 0.4377433	total: 8.91s	remaining: 25.4s
325:	learn: 0.4377103	total: 8.94s	remaining: 25.3s
326:	learn: 0.4376358	total: 8.97s	remaining: 25.3s
327:	learn: 0.4375970	total: 9.01s	remaining: 25.3s
328:	learn: 0.4375457	total: 9.04s	remaining: 25.3s
329:	learn: 0.4374962	total: 9.07s	remaining: 25.3s
330:	learn: 0.4374563	total: 9.11s	remaining: 25.3s
331:	learn: 0.4374305	total: 9.12s	remaining: 25.2s
332:	learn: 0.4373772	total: 9.15s	remaining: 25.2s
333:	learn: 0.4373425	total: 9.17s	remaining: 25.2s
334:	learn: 0.4373099	total: 9.2s	remaining: 25.1s
335:	learn: 0.4372548	total: 9.22s	remaining: 25.1s
336:	learn: 0.4372043	total: 9.24s	remaining: 25s
337:	learn: 0.4371546	total: 9.27s	remaining: 25s
338:	learn: 0.4371119	total: 9.3s	remaining: 25s
339:	learn: 0.4370794	total: 9.32s	remaining: 25s
340:	learn: 0.4370359	total: 9.35s	remaining: 24.9s
341:	learn: 0.4369890	total: 9.39s	remaining: 24.9s
342:	learn: 0.4369429	total: 9.43s	remaining: 24.9s
343:	learn: 0.4369048	

484:	learn: 0.4316852	total: 13.2s	remaining: 20.8s
485:	learn: 0.4316432	total: 13.2s	remaining: 20.8s
486:	learn: 0.4316135	total: 13.2s	remaining: 20.7s
487:	learn: 0.4315821	total: 13.3s	remaining: 20.7s
488:	learn: 0.4315604	total: 13.3s	remaining: 20.7s
489:	learn: 0.4315472	total: 13.3s	remaining: 20.6s
490:	learn: 0.4315245	total: 13.3s	remaining: 20.6s
491:	learn: 0.4314880	total: 13.4s	remaining: 20.6s
492:	learn: 0.4314425	total: 13.4s	remaining: 20.6s
493:	learn: 0.4314010	total: 13.4s	remaining: 20.6s
494:	learn: 0.4313636	total: 13.5s	remaining: 20.5s
495:	learn: 0.4313088	total: 13.5s	remaining: 20.6s
496:	learn: 0.4312855	total: 13.6s	remaining: 20.6s
497:	learn: 0.4312535	total: 13.6s	remaining: 20.6s
498:	learn: 0.4312236	total: 13.6s	remaining: 20.5s
499:	learn: 0.4311870	total: 13.7s	remaining: 20.5s
500:	learn: 0.4311458	total: 13.7s	remaining: 20.5s
501:	learn: 0.4311128	total: 13.7s	remaining: 20.5s
502:	learn: 0.4310826	total: 13.8s	remaining: 20.5s
503:	learn: 

645:	learn: 0.4269455	total: 17.4s	remaining: 16.3s
646:	learn: 0.4269217	total: 17.5s	remaining: 16.3s
647:	learn: 0.4268872	total: 17.5s	remaining: 16.2s
648:	learn: 0.4268674	total: 17.5s	remaining: 16.2s
649:	learn: 0.4268459	total: 17.5s	remaining: 16.2s
650:	learn: 0.4268306	total: 17.6s	remaining: 16.1s
651:	learn: 0.4268107	total: 17.6s	remaining: 16.1s
652:	learn: 0.4267904	total: 17.6s	remaining: 16.1s
653:	learn: 0.4267590	total: 17.6s	remaining: 16.1s
654:	learn: 0.4267393	total: 17.7s	remaining: 16s
655:	learn: 0.4267080	total: 17.7s	remaining: 16s
656:	learn: 0.4266717	total: 17.7s	remaining: 16s
657:	learn: 0.4266597	total: 17.7s	remaining: 16s
658:	learn: 0.4266342	total: 17.8s	remaining: 15.9s
659:	learn: 0.4266006	total: 17.8s	remaining: 15.9s
660:	learn: 0.4265653	total: 17.8s	remaining: 15.9s
661:	learn: 0.4265407	total: 17.8s	remaining: 15.8s
662:	learn: 0.4265155	total: 17.9s	remaining: 15.8s
663:	learn: 0.4264908	total: 17.9s	remaining: 15.8s
664:	learn: 0.426454

805:	learn: 0.4224947	total: 21.6s	remaining: 11.9s
806:	learn: 0.4224637	total: 21.6s	remaining: 11.9s
807:	learn: 0.4224342	total: 21.7s	remaining: 11.9s
808:	learn: 0.4224106	total: 21.7s	remaining: 11.8s
809:	learn: 0.4223810	total: 21.7s	remaining: 11.8s
810:	learn: 0.4223450	total: 21.8s	remaining: 11.8s
811:	learn: 0.4223181	total: 21.8s	remaining: 11.8s
812:	learn: 0.4222892	total: 21.8s	remaining: 11.7s
813:	learn: 0.4222612	total: 21.9s	remaining: 11.7s
814:	learn: 0.4222411	total: 21.9s	remaining: 11.7s
815:	learn: 0.4222105	total: 21.9s	remaining: 11.7s
816:	learn: 0.4221838	total: 21.9s	remaining: 11.6s
817:	learn: 0.4221585	total: 22s	remaining: 11.6s
818:	learn: 0.4221328	total: 22s	remaining: 11.6s
819:	learn: 0.4221124	total: 22s	remaining: 11.5s
820:	learn: 0.4220721	total: 22s	remaining: 11.5s
821:	learn: 0.4220495	total: 22.1s	remaining: 11.5s
822:	learn: 0.4220205	total: 22.1s	remaining: 11.5s
823:	learn: 0.4219857	total: 22.1s	remaining: 11.4s
824:	learn: 0.421957

968:	learn: 0.4173643	total: 26.4s	remaining: 7.67s
969:	learn: 0.4173324	total: 26.5s	remaining: 7.64s
970:	learn: 0.4172875	total: 26.5s	remaining: 7.61s
971:	learn: 0.4172491	total: 26.5s	remaining: 7.59s
972:	learn: 0.4172171	total: 26.6s	remaining: 7.56s
973:	learn: 0.4171764	total: 26.6s	remaining: 7.53s
974:	learn: 0.4171417	total: 26.6s	remaining: 7.51s
975:	learn: 0.4171085	total: 26.7s	remaining: 7.49s
976:	learn: 0.4170737	total: 26.7s	remaining: 7.46s
977:	learn: 0.4170360	total: 26.7s	remaining: 7.43s
978:	learn: 0.4170045	total: 26.8s	remaining: 7.41s
979:	learn: 0.4169562	total: 26.8s	remaining: 7.38s
980:	learn: 0.4169203	total: 26.8s	remaining: 7.35s
981:	learn: 0.4168909	total: 26.9s	remaining: 7.33s
982:	learn: 0.4168620	total: 26.9s	remaining: 7.3s
983:	learn: 0.4168308	total: 26.9s	remaining: 7.28s
984:	learn: 0.4167961	total: 26.9s	remaining: 7.25s
985:	learn: 0.4167531	total: 27s	remaining: 7.22s
986:	learn: 0.4167087	total: 27s	remaining: 7.2s
987:	learn: 0.4166

1131:	learn: 0.4115298	total: 31.1s	remaining: 3.24s
1132:	learn: 0.4114979	total: 31.1s	remaining: 3.21s
1133:	learn: 0.4114684	total: 31.1s	remaining: 3.18s
1134:	learn: 0.4114249	total: 31.1s	remaining: 3.16s
1135:	learn: 0.4113902	total: 31.2s	remaining: 3.13s
1136:	learn: 0.4113548	total: 31.2s	remaining: 3.1s
1137:	learn: 0.4113223	total: 31.2s	remaining: 3.07s
1138:	learn: 0.4112915	total: 31.3s	remaining: 3.05s
1139:	learn: 0.4112583	total: 31.3s	remaining: 3.02s
1140:	learn: 0.4112240	total: 31.3s	remaining: 2.99s
1141:	learn: 0.4111900	total: 31.3s	remaining: 2.96s
1142:	learn: 0.4111483	total: 31.4s	remaining: 2.94s
1143:	learn: 0.4111055	total: 31.4s	remaining: 2.91s
1144:	learn: 0.4110649	total: 31.4s	remaining: 2.88s
1145:	learn: 0.4110338	total: 31.5s	remaining: 2.85s
1146:	learn: 0.4110044	total: 31.5s	remaining: 2.83s
1147:	learn: 0.4109696	total: 31.5s	remaining: 2.8s
1148:	learn: 0.4109324	total: 31.6s	remaining: 2.77s
1149:	learn: 0.4108936	total: 31.6s	remaining: 2

In [46]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

preds = (preds >= 0.5).astype(int)
print("F1_score: ", f1_score(y_test, preds))
#empeoró el resultado

RMSE: 0.419799
F1_score:  0.6537102473498234


In [47]:
#Pruebo con CatBoostClassifier
#CatboostClassifier por defecto
model = CatBoostClassifier(random_seed = 41, one_hot_max_size = 3500)
model.fit(X_train, y_train, cat_features = [0,1])
preds = model.predict(X_test)

Learning rate set to 0.022283
0:	learn: 0.6884811	total: 19.1ms	remaining: 19.1s
1:	learn: 0.6840171	total: 36.1ms	remaining: 18s
2:	learn: 0.6800289	total: 55.4ms	remaining: 18.4s
3:	learn: 0.6759512	total: 72.7ms	remaining: 18.1s
4:	learn: 0.6730145	total: 79.3ms	remaining: 15.8s
5:	learn: 0.6696386	total: 97.3ms	remaining: 16.1s
6:	learn: 0.6659784	total: 115ms	remaining: 16.3s
7:	learn: 0.6629749	total: 133ms	remaining: 16.5s
8:	learn: 0.6596056	total: 150ms	remaining: 16.5s
9:	learn: 0.6565956	total: 167ms	remaining: 16.6s
10:	learn: 0.6536319	total: 187ms	remaining: 16.8s
11:	learn: 0.6507997	total: 208ms	remaining: 17.1s
12:	learn: 0.6477470	total: 226ms	remaining: 17.2s
13:	learn: 0.6450815	total: 244ms	remaining: 17.2s
14:	learn: 0.6427168	total: 261ms	remaining: 17.1s
15:	learn: 0.6405177	total: 278ms	remaining: 17.1s
16:	learn: 0.6384265	total: 290ms	remaining: 16.8s
17:	learn: 0.6360762	total: 307ms	remaining: 16.8s
18:	learn: 0.6340800	total: 324ms	remaining: 16.7s
19:	lea

168:	learn: 0.5562251	total: 3.21s	remaining: 15.8s
169:	learn: 0.5559972	total: 3.23s	remaining: 15.8s
170:	learn: 0.5558943	total: 3.25s	remaining: 15.8s
171:	learn: 0.5556511	total: 3.27s	remaining: 15.8s
172:	learn: 0.5553986	total: 3.29s	remaining: 15.8s
173:	learn: 0.5551091	total: 3.31s	remaining: 15.7s
174:	learn: 0.5549243	total: 3.33s	remaining: 15.7s
175:	learn: 0.5546631	total: 3.35s	remaining: 15.7s
176:	learn: 0.5543499	total: 3.37s	remaining: 15.7s
177:	learn: 0.5540967	total: 3.39s	remaining: 15.6s
178:	learn: 0.5538344	total: 3.41s	remaining: 15.6s
179:	learn: 0.5536507	total: 3.43s	remaining: 15.6s
180:	learn: 0.5534994	total: 3.44s	remaining: 15.6s
181:	learn: 0.5532922	total: 3.46s	remaining: 15.6s
182:	learn: 0.5531204	total: 3.48s	remaining: 15.5s
183:	learn: 0.5529860	total: 3.5s	remaining: 15.5s
184:	learn: 0.5528259	total: 3.52s	remaining: 15.5s
185:	learn: 0.5527228	total: 3.53s	remaining: 15.5s
186:	learn: 0.5525275	total: 3.55s	remaining: 15.4s
187:	learn: 0

334:	learn: 0.5288417	total: 6.53s	remaining: 13s
335:	learn: 0.5286653	total: 6.55s	remaining: 12.9s
336:	learn: 0.5285018	total: 6.57s	remaining: 12.9s
337:	learn: 0.5282892	total: 6.58s	remaining: 12.9s
338:	learn: 0.5281632	total: 6.6s	remaining: 12.9s
339:	learn: 0.5280422	total: 6.62s	remaining: 12.9s
340:	learn: 0.5278664	total: 6.64s	remaining: 12.8s
341:	learn: 0.5277097	total: 6.66s	remaining: 12.8s
342:	learn: 0.5275934	total: 6.67s	remaining: 12.8s
343:	learn: 0.5273938	total: 6.68s	remaining: 12.7s
344:	learn: 0.5271785	total: 6.7s	remaining: 12.7s
345:	learn: 0.5270232	total: 6.72s	remaining: 12.7s
346:	learn: 0.5267483	total: 6.74s	remaining: 12.7s
347:	learn: 0.5265244	total: 6.76s	remaining: 12.7s
348:	learn: 0.5264131	total: 6.78s	remaining: 12.6s
349:	learn: 0.5262502	total: 6.8s	remaining: 12.6s
350:	learn: 0.5261069	total: 6.83s	remaining: 12.6s
351:	learn: 0.5259727	total: 6.87s	remaining: 12.6s
352:	learn: 0.5258075	total: 6.9s	remaining: 12.7s
353:	learn: 0.5256

497:	learn: 0.5002806	total: 10.2s	remaining: 10.3s
498:	learn: 0.5000995	total: 10.2s	remaining: 10.2s
499:	learn: 0.4999292	total: 10.2s	remaining: 10.2s
500:	learn: 0.4997956	total: 10.2s	remaining: 10.2s
501:	learn: 0.4996085	total: 10.3s	remaining: 10.2s
502:	learn: 0.4994412	total: 10.3s	remaining: 10.2s
503:	learn: 0.4992756	total: 10.3s	remaining: 10.1s
504:	learn: 0.4991261	total: 10.3s	remaining: 10.1s
505:	learn: 0.4989699	total: 10.3s	remaining: 10.1s
506:	learn: 0.4987842	total: 10.3s	remaining: 10.1s
507:	learn: 0.4986060	total: 10.4s	remaining: 10s
508:	learn: 0.4984353	total: 10.4s	remaining: 10s
509:	learn: 0.4983055	total: 10.4s	remaining: 9.99s
510:	learn: 0.4981481	total: 10.4s	remaining: 9.98s
511:	learn: 0.4979484	total: 10.4s	remaining: 9.96s
512:	learn: 0.4977519	total: 10.5s	remaining: 9.94s
513:	learn: 0.4975913	total: 10.5s	remaining: 9.93s
514:	learn: 0.4973321	total: 10.5s	remaining: 9.91s
515:	learn: 0.4972164	total: 10.5s	remaining: 9.89s
516:	learn: 0.49

663:	learn: 0.4739699	total: 13.3s	remaining: 6.75s
664:	learn: 0.4738419	total: 13.4s	remaining: 6.73s
665:	learn: 0.4736746	total: 13.4s	remaining: 6.71s
666:	learn: 0.4735454	total: 13.4s	remaining: 6.69s
667:	learn: 0.4734118	total: 13.4s	remaining: 6.67s
668:	learn: 0.4732599	total: 13.5s	remaining: 6.66s
669:	learn: 0.4731240	total: 13.5s	remaining: 6.64s
670:	learn: 0.4729907	total: 13.5s	remaining: 6.62s
671:	learn: 0.4728558	total: 13.5s	remaining: 6.6s
672:	learn: 0.4727024	total: 13.6s	remaining: 6.58s
673:	learn: 0.4724800	total: 13.6s	remaining: 6.57s
674:	learn: 0.4723678	total: 13.6s	remaining: 6.54s
675:	learn: 0.4723168	total: 13.6s	remaining: 6.52s
676:	learn: 0.4721573	total: 13.6s	remaining: 6.5s
677:	learn: 0.4720186	total: 13.6s	remaining: 6.48s
678:	learn: 0.4718948	total: 13.7s	remaining: 6.46s
679:	learn: 0.4717731	total: 13.7s	remaining: 6.43s
680:	learn: 0.4716360	total: 13.7s	remaining: 6.41s
681:	learn: 0.4715845	total: 13.7s	remaining: 6.39s
682:	learn: 0.

826:	learn: 0.4527737	total: 16.5s	remaining: 3.46s
827:	learn: 0.4526489	total: 16.6s	remaining: 3.44s
828:	learn: 0.4524883	total: 16.6s	remaining: 3.42s
829:	learn: 0.4524484	total: 16.6s	remaining: 3.4s
830:	learn: 0.4523373	total: 16.6s	remaining: 3.38s
831:	learn: 0.4522230	total: 16.6s	remaining: 3.36s
832:	learn: 0.4520814	total: 16.7s	remaining: 3.34s
833:	learn: 0.4519379	total: 16.7s	remaining: 3.32s
834:	learn: 0.4518067	total: 16.7s	remaining: 3.3s
835:	learn: 0.4516712	total: 16.7s	remaining: 3.28s
836:	learn: 0.4516318	total: 16.7s	remaining: 3.26s
837:	learn: 0.4515164	total: 16.8s	remaining: 3.24s
838:	learn: 0.4513662	total: 16.8s	remaining: 3.22s
839:	learn: 0.4512742	total: 16.8s	remaining: 3.2s
840:	learn: 0.4511630	total: 16.8s	remaining: 3.18s
841:	learn: 0.4510329	total: 16.8s	remaining: 3.16s
842:	learn: 0.4509155	total: 16.8s	remaining: 3.13s
843:	learn: 0.4508766	total: 16.9s	remaining: 3.12s
844:	learn: 0.4507666	total: 16.9s	remaining: 3.1s
845:	learn: 0.45

991:	learn: 0.4354901	total: 19.5s	remaining: 158ms
992:	learn: 0.4353943	total: 19.6s	remaining: 138ms
993:	learn: 0.4353615	total: 19.6s	remaining: 118ms
994:	learn: 0.4353345	total: 19.6s	remaining: 98.5ms
995:	learn: 0.4352186	total: 19.6s	remaining: 78.8ms
996:	learn: 0.4350709	total: 19.6s	remaining: 59.1ms
997:	learn: 0.4350405	total: 19.6s	remaining: 39.4ms
998:	learn: 0.4349446	total: 19.7s	remaining: 19.7ms
999:	learn: 0.4349175	total: 19.7s	remaining: 0us


In [48]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

print("F1_score: ", f1_score(y_test, preds))

RMSE: 0.477326
F1_score:  0.7120331950207468


In [50]:
model = CatBoostClassifier(iterations = 1250, learning_rate = 0.01, one_hot_max_size = 3500,
                           depth = 7, random_seed = 41)
model.fit(X_train, y_train, cat_features = [0,1])
preds = model.predict(X_test)

0:	learn: 0.6910371	total: 43.1ms	remaining: 53.8s
1:	learn: 0.6888382	total: 74.8ms	remaining: 46.7s
2:	learn: 0.6867946	total: 102ms	remaining: 42.5s
3:	learn: 0.6847131	total: 130ms	remaining: 40.4s
4:	learn: 0.6829539	total: 158ms	remaining: 39.2s
5:	learn: 0.6813863	total: 166ms	remaining: 34.4s
6:	learn: 0.6796585	total: 193ms	remaining: 34.3s
7:	learn: 0.6777828	total: 220ms	remaining: 34.2s
8:	learn: 0.6762641	total: 235ms	remaining: 32.4s
9:	learn: 0.6742548	total: 266ms	remaining: 33s
10:	learn: 0.6724929	total: 294ms	remaining: 33.1s
11:	learn: 0.6711727	total: 306ms	remaining: 31.6s
12:	learn: 0.6692347	total: 334ms	remaining: 31.8s
13:	learn: 0.6675980	total: 360ms	remaining: 31.8s
14:	learn: 0.6661858	total: 387ms	remaining: 31.9s
15:	learn: 0.6646257	total: 399ms	remaining: 30.8s
16:	learn: 0.6634851	total: 406ms	remaining: 29.4s
17:	learn: 0.6617134	total: 437ms	remaining: 29.9s
18:	learn: 0.6603551	total: 466ms	remaining: 30.2s
19:	learn: 0.6590088	total: 478ms	remaini

161:	learn: 0.5800222	total: 4.44s	remaining: 29.8s
162:	learn: 0.5797263	total: 4.47s	remaining: 29.8s
163:	learn: 0.5795594	total: 4.5s	remaining: 29.8s
164:	learn: 0.5793908	total: 4.53s	remaining: 29.8s
165:	learn: 0.5791805	total: 4.55s	remaining: 29.7s
166:	learn: 0.5789361	total: 4.58s	remaining: 29.7s
167:	learn: 0.5786998	total: 4.61s	remaining: 29.7s
168:	learn: 0.5785419	total: 4.64s	remaining: 29.7s
169:	learn: 0.5784099	total: 4.65s	remaining: 29.6s
170:	learn: 0.5782086	total: 4.68s	remaining: 29.5s
171:	learn: 0.5779548	total: 4.71s	remaining: 29.5s
172:	learn: 0.5777922	total: 4.73s	remaining: 29.5s
173:	learn: 0.5776220	total: 4.76s	remaining: 29.4s
174:	learn: 0.5774260	total: 4.79s	remaining: 29.4s
175:	learn: 0.5771773	total: 4.83s	remaining: 29.4s
176:	learn: 0.5769868	total: 4.85s	remaining: 29.4s
177:	learn: 0.5767837	total: 4.88s	remaining: 29.4s
178:	learn: 0.5765943	total: 4.91s	remaining: 29.4s
179:	learn: 0.5764384	total: 4.94s	remaining: 29.3s
180:	learn: 0

322:	learn: 0.5563493	total: 9.13s	remaining: 26.2s
323:	learn: 0.5562303	total: 9.17s	remaining: 26.2s
324:	learn: 0.5561406	total: 9.26s	remaining: 26.4s
325:	learn: 0.5559947	total: 9.32s	remaining: 26.4s
326:	learn: 0.5558509	total: 9.36s	remaining: 26.4s
327:	learn: 0.5556700	total: 9.46s	remaining: 26.6s
328:	learn: 0.5555355	total: 9.58s	remaining: 26.8s
329:	learn: 0.5554277	total: 9.64s	remaining: 26.9s
330:	learn: 0.5553208	total: 9.71s	remaining: 27s
331:	learn: 0.5552315	total: 9.78s	remaining: 27s
332:	learn: 0.5551393	total: 9.82s	remaining: 27s
333:	learn: 0.5550001	total: 9.87s	remaining: 27.1s
334:	learn: 0.5549210	total: 9.92s	remaining: 27.1s
335:	learn: 0.5547845	total: 10s	remaining: 27.2s
336:	learn: 0.5547142	total: 10.1s	remaining: 27.2s
337:	learn: 0.5546055	total: 10.1s	remaining: 27.3s
338:	learn: 0.5544968	total: 10.2s	remaining: 27.5s
339:	learn: 0.5543722	total: 10.3s	remaining: 27.6s
340:	learn: 0.5542624	total: 10.4s	remaining: 27.7s
341:	learn: 0.554110

483:	learn: 0.5409102	total: 16.6s	remaining: 26.3s
484:	learn: 0.5408370	total: 16.6s	remaining: 26.3s
485:	learn: 0.5407759	total: 16.7s	remaining: 26.2s
486:	learn: 0.5407191	total: 16.7s	remaining: 26.2s
487:	learn: 0.5406357	total: 16.7s	remaining: 26.2s
488:	learn: 0.5405493	total: 16.8s	remaining: 26.1s
489:	learn: 0.5404576	total: 16.8s	remaining: 26.1s
490:	learn: 0.5403647	total: 16.9s	remaining: 26.1s
491:	learn: 0.5402709	total: 16.9s	remaining: 26.1s
492:	learn: 0.5401820	total: 17s	remaining: 26.1s
493:	learn: 0.5400759	total: 17s	remaining: 26s
494:	learn: 0.5400145	total: 17s	remaining: 26s
495:	learn: 0.5399112	total: 17.1s	remaining: 26s
496:	learn: 0.5398301	total: 17.1s	remaining: 25.9s
497:	learn: 0.5397155	total: 17.2s	remaining: 25.9s
498:	learn: 0.5395884	total: 17.2s	remaining: 25.9s
499:	learn: 0.5395027	total: 17.2s	remaining: 25.9s
500:	learn: 0.5393831	total: 17.3s	remaining: 25.8s
501:	learn: 0.5393174	total: 17.3s	remaining: 25.8s
502:	learn: 0.5392569	to

644:	learn: 0.5277678	total: 23.1s	remaining: 21.7s
645:	learn: 0.5276924	total: 23.2s	remaining: 21.6s
646:	learn: 0.5276160	total: 23.2s	remaining: 21.6s
647:	learn: 0.5275522	total: 23.2s	remaining: 21.6s
648:	learn: 0.5274973	total: 23.3s	remaining: 21.5s
649:	learn: 0.5274387	total: 23.3s	remaining: 21.5s
650:	learn: 0.5273858	total: 23.3s	remaining: 21.5s
651:	learn: 0.5273136	total: 23.4s	remaining: 21.4s
652:	learn: 0.5272677	total: 23.4s	remaining: 21.4s
653:	learn: 0.5272111	total: 23.4s	remaining: 21.3s
654:	learn: 0.5271147	total: 23.4s	remaining: 21.3s
655:	learn: 0.5270227	total: 23.5s	remaining: 21.3s
656:	learn: 0.5269569	total: 23.5s	remaining: 21.2s
657:	learn: 0.5268515	total: 23.5s	remaining: 21.2s
658:	learn: 0.5267396	total: 23.6s	remaining: 21.1s
659:	learn: 0.5266666	total: 23.6s	remaining: 21.1s
660:	learn: 0.5265249	total: 23.6s	remaining: 21.1s
661:	learn: 0.5264655	total: 23.7s	remaining: 21s
662:	learn: 0.5264286	total: 23.7s	remaining: 21s
663:	learn: 0.52

806:	learn: 0.5155190	total: 29.5s	remaining: 16.2s
807:	learn: 0.5154205	total: 29.6s	remaining: 16.2s
808:	learn: 0.5153282	total: 29.6s	remaining: 16.2s
809:	learn: 0.5152490	total: 29.7s	remaining: 16.1s
810:	learn: 0.5151920	total: 29.8s	remaining: 16.1s
811:	learn: 0.5150908	total: 29.8s	remaining: 16.1s
812:	learn: 0.5150293	total: 29.9s	remaining: 16s
813:	learn: 0.5149575	total: 29.9s	remaining: 16s
814:	learn: 0.5148664	total: 30s	remaining: 16s
815:	learn: 0.5147753	total: 30s	remaining: 16s
816:	learn: 0.5146974	total: 30.1s	remaining: 15.9s
817:	learn: 0.5146363	total: 30.1s	remaining: 15.9s
818:	learn: 0.5145586	total: 30.2s	remaining: 15.9s
819:	learn: 0.5144541	total: 30.2s	remaining: 15.9s
820:	learn: 0.5143704	total: 30.3s	remaining: 15.8s
821:	learn: 0.5142943	total: 30.3s	remaining: 15.8s
822:	learn: 0.5142163	total: 30.4s	remaining: 15.8s
823:	learn: 0.5141363	total: 30.5s	remaining: 15.7s
824:	learn: 0.5140533	total: 30.5s	remaining: 15.7s
825:	learn: 0.5139887	to

966:	learn: 0.5016503	total: 36s	remaining: 10.5s
967:	learn: 0.5015659	total: 36s	remaining: 10.5s
968:	learn: 0.5014818	total: 36s	remaining: 10.4s
969:	learn: 0.5014064	total: 36.1s	remaining: 10.4s
970:	learn: 0.5013144	total: 36.1s	remaining: 10.4s
971:	learn: 0.5012192	total: 36.1s	remaining: 10.3s
972:	learn: 0.5011311	total: 36.2s	remaining: 10.3s
973:	learn: 0.5010356	total: 36.2s	remaining: 10.3s
974:	learn: 0.5009482	total: 36.3s	remaining: 10.2s
975:	learn: 0.5008536	total: 36.3s	remaining: 10.2s
976:	learn: 0.5007830	total: 36.3s	remaining: 10.2s
977:	learn: 0.5006936	total: 36.4s	remaining: 10.1s
978:	learn: 0.5005912	total: 36.4s	remaining: 10.1s
979:	learn: 0.5005200	total: 36.5s	remaining: 10.1s
980:	learn: 0.5004256	total: 36.6s	remaining: 10s
981:	learn: 0.5003306	total: 36.6s	remaining: 9.99s
982:	learn: 0.5002300	total: 36.7s	remaining: 9.98s
983:	learn: 0.5001480	total: 36.8s	remaining: 9.94s
984:	learn: 0.5000732	total: 36.8s	remaining: 9.91s
985:	learn: 0.499951

1124:	learn: 0.4865133	total: 43.6s	remaining: 4.84s
1125:	learn: 0.4864253	total: 43.6s	remaining: 4.8s
1126:	learn: 0.4863428	total: 43.6s	remaining: 4.76s
1127:	learn: 0.4862460	total: 43.7s	remaining: 4.72s
1128:	learn: 0.4861837	total: 43.7s	remaining: 4.68s
1129:	learn: 0.4860773	total: 43.7s	remaining: 4.64s
1130:	learn: 0.4859976	total: 43.8s	remaining: 4.6s
1131:	learn: 0.4859020	total: 43.8s	remaining: 4.57s
1132:	learn: 0.4858273	total: 43.9s	remaining: 4.53s
1133:	learn: 0.4857233	total: 43.9s	remaining: 4.49s
1134:	learn: 0.4856575	total: 43.9s	remaining: 4.45s
1135:	learn: 0.4855461	total: 44s	remaining: 4.41s
1136:	learn: 0.4854688	total: 44s	remaining: 4.37s
1137:	learn: 0.4853517	total: 44s	remaining: 4.33s
1138:	learn: 0.4852427	total: 44s	remaining: 4.29s
1139:	learn: 0.4851394	total: 44.1s	remaining: 4.25s
1140:	learn: 0.4850397	total: 44.1s	remaining: 4.21s
1141:	learn: 0.4849495	total: 44.1s	remaining: 4.17s
1142:	learn: 0.4848629	total: 44.2s	remaining: 4.13s
114

In [51]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

print("F1_score: ", f1_score(y_test, preds))
#empeoró el resultado

RMSE: 0.496871
F1_score:  0.6769759450171821


In [52]:
model = CatBoostClassifier(iterations = 1500, learning_rate = 0.01, one_hot_max_size = 3500,
                           depth = 9, random_seed = 41)
model.fit(X_train, y_train, cat_features = [0,1])
preds = model.predict(X_test)

0:	learn: 0.6910371	total: 19.2ms	remaining: 28.7s
1:	learn: 0.6886459	total: 143ms	remaining: 1m 47s
2:	learn: 0.6862966	total: 237ms	remaining: 1m 58s
3:	learn: 0.6840279	total: 335ms	remaining: 2m 5s
4:	learn: 0.6819189	total: 424ms	remaining: 2m 6s
5:	learn: 0.6800392	total: 445ms	remaining: 1m 50s
6:	learn: 0.6780143	total: 548ms	remaining: 1m 56s
7:	learn: 0.6759769	total: 605ms	remaining: 1m 52s
8:	learn: 0.6740374	total: 708ms	remaining: 1m 57s
9:	learn: 0.6722552	total: 740ms	remaining: 1m 50s
10:	learn: 0.6702940	total: 837ms	remaining: 1m 53s
11:	learn: 0.6684543	total: 1s	remaining: 2m 4s
12:	learn: 0.6666352	total: 1.07s	remaining: 2m 2s
13:	learn: 0.6650147	total: 1.14s	remaining: 2m
14:	learn: 0.6636120	total: 1.23s	remaining: 2m 1s
15:	learn: 0.6617615	total: 1.33s	remaining: 2m 3s
16:	learn: 0.6601983	total: 1.38s	remaining: 2m
17:	learn: 0.6585823	total: 1.48s	remaining: 2m 2s
18:	learn: 0.6569309	total: 1.6s	remaining: 2m 4s
19:	learn: 0.6554095	total: 1.72s	remainin

160:	learn: 0.5705240	total: 12.5s	remaining: 1m 43s
161:	learn: 0.5702694	total: 12.6s	remaining: 1m 43s
162:	learn: 0.5700608	total: 12.7s	remaining: 1m 44s
163:	learn: 0.5698434	total: 12.8s	remaining: 1m 44s
164:	learn: 0.5696330	total: 12.9s	remaining: 1m 44s
165:	learn: 0.5693541	total: 13s	remaining: 1m 44s
166:	learn: 0.5691720	total: 13.1s	remaining: 1m 44s
167:	learn: 0.5688359	total: 13.3s	remaining: 1m 45s
168:	learn: 0.5684627	total: 13.4s	remaining: 1m 45s
169:	learn: 0.5682420	total: 13.5s	remaining: 1m 45s
170:	learn: 0.5679563	total: 13.6s	remaining: 1m 45s
171:	learn: 0.5676826	total: 13.7s	remaining: 1m 45s
172:	learn: 0.5675184	total: 13.8s	remaining: 1m 45s
173:	learn: 0.5672660	total: 13.9s	remaining: 1m 45s
174:	learn: 0.5670170	total: 14s	remaining: 1m 46s
175:	learn: 0.5667991	total: 14.1s	remaining: 1m 46s
176:	learn: 0.5666123	total: 14.2s	remaining: 1m 46s
177:	learn: 0.5663519	total: 14.3s	remaining: 1m 46s
178:	learn: 0.5661127	total: 14.4s	remaining: 1m 4

316:	learn: 0.5433281	total: 27.9s	remaining: 1m 44s
317:	learn: 0.5431412	total: 28s	remaining: 1m 44s
318:	learn: 0.5429876	total: 28.1s	remaining: 1m 44s
319:	learn: 0.5428168	total: 28.2s	remaining: 1m 44s
320:	learn: 0.5426689	total: 28.3s	remaining: 1m 43s
321:	learn: 0.5424825	total: 28.4s	remaining: 1m 43s
322:	learn: 0.5423253	total: 28.5s	remaining: 1m 43s
323:	learn: 0.5421843	total: 28.6s	remaining: 1m 43s
324:	learn: 0.5420161	total: 28.7s	remaining: 1m 43s
325:	learn: 0.5419273	total: 28.7s	remaining: 1m 43s
326:	learn: 0.5417961	total: 28.8s	remaining: 1m 43s
327:	learn: 0.5417367	total: 28.8s	remaining: 1m 43s
328:	learn: 0.5415964	total: 28.9s	remaining: 1m 42s
329:	learn: 0.5414836	total: 29s	remaining: 1m 42s
330:	learn: 0.5412544	total: 29.1s	remaining: 1m 42s
331:	learn: 0.5411320	total: 29.1s	remaining: 1m 42s
332:	learn: 0.5409757	total: 29.2s	remaining: 1m 42s
333:	learn: 0.5408505	total: 29.3s	remaining: 1m 42s
334:	learn: 0.5407212	total: 29.4s	remaining: 1m 4

474:	learn: 0.5253255	total: 41.4s	remaining: 1m 29s
475:	learn: 0.5252199	total: 41.5s	remaining: 1m 29s
476:	learn: 0.5250969	total: 41.6s	remaining: 1m 29s
477:	learn: 0.5250518	total: 41.7s	remaining: 1m 29s
478:	learn: 0.5250265	total: 41.7s	remaining: 1m 28s
479:	learn: 0.5249108	total: 41.8s	remaining: 1m 28s
480:	learn: 0.5248091	total: 41.9s	remaining: 1m 28s
481:	learn: 0.5247246	total: 41.9s	remaining: 1m 28s
482:	learn: 0.5246133	total: 42s	remaining: 1m 28s
483:	learn: 0.5244443	total: 42.1s	remaining: 1m 28s
484:	learn: 0.5242980	total: 42.2s	remaining: 1m 28s
485:	learn: 0.5241925	total: 42.3s	remaining: 1m 28s
486:	learn: 0.5240824	total: 42.4s	remaining: 1m 28s
487:	learn: 0.5239888	total: 42.5s	remaining: 1m 28s
488:	learn: 0.5238817	total: 42.6s	remaining: 1m 28s
489:	learn: 0.5237757	total: 42.7s	remaining: 1m 27s
490:	learn: 0.5236805	total: 42.8s	remaining: 1m 27s
491:	learn: 0.5236143	total: 42.8s	remaining: 1m 27s
492:	learn: 0.5235552	total: 42.8s	remaining: 1m

630:	learn: 0.5109334	total: 53.9s	remaining: 1m 14s
631:	learn: 0.5108513	total: 54s	remaining: 1m 14s
632:	learn: 0.5107793	total: 54.1s	remaining: 1m 14s
633:	learn: 0.5107089	total: 54.2s	remaining: 1m 14s
634:	learn: 0.5106008	total: 54.3s	remaining: 1m 13s
635:	learn: 0.5105076	total: 54.4s	remaining: 1m 13s
636:	learn: 0.5104304	total: 54.5s	remaining: 1m 13s
637:	learn: 0.5103639	total: 54.6s	remaining: 1m 13s
638:	learn: 0.5102509	total: 54.6s	remaining: 1m 13s
639:	learn: 0.5101560	total: 54.7s	remaining: 1m 13s
640:	learn: 0.5100813	total: 54.8s	remaining: 1m 13s
641:	learn: 0.5100004	total: 54.9s	remaining: 1m 13s
642:	learn: 0.5099237	total: 55s	remaining: 1m 13s
643:	learn: 0.5098546	total: 55.1s	remaining: 1m 13s
644:	learn: 0.5097674	total: 55.2s	remaining: 1m 13s
645:	learn: 0.5096429	total: 55.3s	remaining: 1m 13s
646:	learn: 0.5095615	total: 55.4s	remaining: 1m 13s
647:	learn: 0.5094492	total: 55.5s	remaining: 1m 12s
648:	learn: 0.5093831	total: 55.6s	remaining: 1m 1

789:	learn: 0.4963512	total: 1m 7s	remaining: 1m
790:	learn: 0.4962555	total: 1m 7s	remaining: 1m
791:	learn: 0.4961623	total: 1m 8s	remaining: 1m
792:	learn: 0.4960602	total: 1m 8s	remaining: 1m
793:	learn: 0.4959971	total: 1m 8s	remaining: 1m
794:	learn: 0.4959548	total: 1m 8s	remaining: 1m
795:	learn: 0.4958767	total: 1m 8s	remaining: 1m
796:	learn: 0.4958198	total: 1m 8s	remaining: 1m
797:	learn: 0.4956866	total: 1m 8s	remaining: 1m
798:	learn: 0.4955775	total: 1m 8s	remaining: 1m
799:	learn: 0.4954832	total: 1m 8s	remaining: 1m
800:	learn: 0.4954156	total: 1m 8s	remaining: 60s
801:	learn: 0.4953181	total: 1m 8s	remaining: 59.9s
802:	learn: 0.4952289	total: 1m 8s	remaining: 59.8s
803:	learn: 0.4951674	total: 1m 8s	remaining: 59.7s
804:	learn: 0.4950635	total: 1m 9s	remaining: 59.6s
805:	learn: 0.4950004	total: 1m 9s	remaining: 59.5s
806:	learn: 0.4948779	total: 1m 9s	remaining: 59.5s
807:	learn: 0.4947732	total: 1m 9s	remaining: 59.4s
808:	learn: 0.4946575	total: 1m 9s	remaining: 5

946:	learn: 0.4810227	total: 1m 21s	remaining: 47.6s
947:	learn: 0.4809138	total: 1m 21s	remaining: 47.5s
948:	learn: 0.4807910	total: 1m 21s	remaining: 47.4s
949:	learn: 0.4806679	total: 1m 21s	remaining: 47.3s
950:	learn: 0.4805653	total: 1m 21s	remaining: 47.3s
951:	learn: 0.4804918	total: 1m 21s	remaining: 47.2s
952:	learn: 0.4803881	total: 1m 22s	remaining: 47.1s
953:	learn: 0.4802787	total: 1m 22s	remaining: 47s
954:	learn: 0.4801441	total: 1m 22s	remaining: 46.9s
955:	learn: 0.4800124	total: 1m 22s	remaining: 46.8s
956:	learn: 0.4799063	total: 1m 22s	remaining: 46.8s
957:	learn: 0.4798232	total: 1m 22s	remaining: 46.7s
958:	learn: 0.4797002	total: 1m 22s	remaining: 46.6s
959:	learn: 0.4795940	total: 1m 22s	remaining: 46.5s
960:	learn: 0.4794846	total: 1m 22s	remaining: 46.4s
961:	learn: 0.4793656	total: 1m 22s	remaining: 46.3s
962:	learn: 0.4792569	total: 1m 22s	remaining: 46.3s
963:	learn: 0.4791339	total: 1m 23s	remaining: 46.2s
964:	learn: 0.4789934	total: 1m 23s	remaining: 4

1100:	learn: 0.4634356	total: 1m 35s	remaining: 34.6s
1101:	learn: 0.4633299	total: 1m 35s	remaining: 34.5s
1102:	learn: 0.4632168	total: 1m 35s	remaining: 34.4s
1103:	learn: 0.4631175	total: 1m 35s	remaining: 34.3s
1104:	learn: 0.4629855	total: 1m 35s	remaining: 34.2s
1105:	learn: 0.4628896	total: 1m 35s	remaining: 34.1s
1106:	learn: 0.4627878	total: 1m 35s	remaining: 34.1s
1107:	learn: 0.4626961	total: 1m 36s	remaining: 34s
1108:	learn: 0.4625831	total: 1m 36s	remaining: 33.9s
1109:	learn: 0.4624622	total: 1m 36s	remaining: 33.8s
1110:	learn: 0.4623519	total: 1m 36s	remaining: 33.7s
1111:	learn: 0.4622166	total: 1m 36s	remaining: 33.6s
1112:	learn: 0.4621031	total: 1m 36s	remaining: 33.6s
1113:	learn: 0.4619753	total: 1m 36s	remaining: 33.5s
1114:	learn: 0.4618569	total: 1m 36s	remaining: 33.4s
1115:	learn: 0.4617268	total: 1m 36s	remaining: 33.3s
1116:	learn: 0.4616153	total: 1m 36s	remaining: 33.2s
1117:	learn: 0.4615103	total: 1m 36s	remaining: 33.1s
1118:	learn: 0.4614303	total: 

1254:	learn: 0.4472595	total: 1m 49s	remaining: 21.4s
1255:	learn: 0.4471472	total: 1m 49s	remaining: 21.3s
1256:	learn: 0.4470649	total: 1m 49s	remaining: 21.2s
1257:	learn: 0.4469629	total: 1m 49s	remaining: 21.1s
1258:	learn: 0.4468707	total: 1m 49s	remaining: 21s
1259:	learn: 0.4467557	total: 1m 49s	remaining: 20.9s
1260:	learn: 0.4466756	total: 1m 49s	remaining: 20.8s
1261:	learn: 0.4465758	total: 1m 50s	remaining: 20.7s
1262:	learn: 0.4464814	total: 1m 50s	remaining: 20.7s
1263:	learn: 0.4463947	total: 1m 50s	remaining: 20.6s
1264:	learn: 0.4462934	total: 1m 50s	remaining: 20.5s
1265:	learn: 0.4461970	total: 1m 50s	remaining: 20.4s
1266:	learn: 0.4461052	total: 1m 50s	remaining: 20.3s
1267:	learn: 0.4460343	total: 1m 50s	remaining: 20.2s
1268:	learn: 0.4459429	total: 1m 50s	remaining: 20.1s
1269:	learn: 0.4458673	total: 1m 50s	remaining: 20.1s
1270:	learn: 0.4457444	total: 1m 50s	remaining: 20s
1271:	learn: 0.4456649	total: 1m 50s	remaining: 19.9s
1272:	learn: 0.4456339	total: 1m

1410:	learn: 0.4329425	total: 2m 3s	remaining: 7.82s
1411:	learn: 0.4328563	total: 2m 4s	remaining: 7.73s
1412:	learn: 0.4327655	total: 2m 4s	remaining: 7.64s
1413:	learn: 0.4326681	total: 2m 4s	remaining: 7.56s
1414:	learn: 0.4325689	total: 2m 4s	remaining: 7.47s
1415:	learn: 0.4324946	total: 2m 4s	remaining: 7.38s
1416:	learn: 0.4323985	total: 2m 4s	remaining: 7.29s
1417:	learn: 0.4323103	total: 2m 4s	remaining: 7.21s
1418:	learn: 0.4322147	total: 2m 4s	remaining: 7.12s
1419:	learn: 0.4321273	total: 2m 4s	remaining: 7.04s
1420:	learn: 0.4320570	total: 2m 4s	remaining: 6.95s
1421:	learn: 0.4319591	total: 2m 5s	remaining: 6.86s
1422:	learn: 0.4319329	total: 2m 5s	remaining: 6.78s
1423:	learn: 0.4318452	total: 2m 5s	remaining: 6.69s
1424:	learn: 0.4317588	total: 2m 5s	remaining: 6.6s
1425:	learn: 0.4316586	total: 2m 5s	remaining: 6.51s
1426:	learn: 0.4315716	total: 2m 5s	remaining: 6.43s
1427:	learn: 0.4314372	total: 2m 5s	remaining: 6.34s
1428:	learn: 0.4312998	total: 2m 5s	remaining: 

In [53]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

print("F1_score: ", f1_score(y_test, preds))
#empeoró el resultado

RMSE: 0.479385
F1_score:  0.7053872053872055
